# 6. – Boosting (XGBoost, LightGBM, CatBoost)

## Gradient Boosting Algorithms

<img src='https://cdn-images-1.medium.com/max/2000/1*i0CA9ho0WArOj-0UdpuKGQ.png' aligh='center'>

The common ensemble techniques like random forests rely on simple averaging of models in the ensemble. The family of boosting methods is based on a different, constructive strategy of ensemble formation. The main idea of boosting is to add new models to the ensemble sequentially. <br> 

In gradient boosting  the learning procedure consecutively fits new models to provide a more accurate estimate of the response variable. The principle idea behind this algorithm is to construct the new base-learners to be maximally correlated with the negative gradient of the loss function, associated with the whole ensemble. Simply put, the next algorithm tries to fix the error of the previous one. 

### Friedman's classic GBM algorithm

So, now we can finally define classic GBM algorithm, suggested by Jerome Friedman in 1999. It is superved algorithm that has the following components:

- dataset $\large \left\{ (x_i, y_i) \right\}_{i=1, \ldots,n}$;
- number of iterations $\large M$;
- choice of loss function $\large L(y, f)$ with defined gradient;
- choice of function family of base algorithms $\large h(x, \theta)$, with the procedure of its training;
- Additional hyperparameters $\large h(x, \theta)$, for example, tree's depth of decision trees;

The only thing left is the initial approximation $\large f_0(x)$. For simplicity, for an initial approximation a constant value $\large \gamma$ is used. The constant value, as well as optimal coefficient $\large \rho $, are retrieved by binary search or by another line search algorithm regarding initial loss function (not a gradient). So, GBM algorithm:

1. Initialize GBM with constant value $\large \hat{f}(x) = \hat{f}_0, \hat{f}_0 = \gamma,  \gamma \in \mathbb{R}$
$\large \hat{f}_0 = \underset{\gamma}{\arg\min} \ \sum_{i = 1}^{n} L(y_i, \gamma)$
2. For each iteration $\large t = 1, \dots, M$ repeat:
1. Count pseudo-residues $\large r_t$
$\large r_{it} = -\left[\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}\right]_{f(x)=\hat{f}(x)}, \quad \mbox{for } i=1,\ldots,n$
2. Build new base algorithm $\large h_t(x)$ as regression on pseudo-residues $\large \left\{ (x_i, r_{it}) \right\}_{i=1, \ldots,n}$
3. Find optimal coefficient $\large \rho_t $ at $\large h_t(x)$ regarding initial loss function
$\large \rho_t = \underset{\rho}{\arg\min} \ \sum_{i = 1}^{n} L(y_i, \hat{f}(x_i) +  \rho \cdot h(x_i, \theta))$
4. Save $\large \hat{f_t}(x) = \rho_t \cdot h_t(x)$
5. Update current approximation $\large \hat{f}(x)$
$\large \hat{f}(x) \leftarrow \hat{f}(x) + \hat{f_t}(x) = \sum_{i = 0}^{t} \hat{f_i}(x)$
3. Compose final GBM model $\large \hat{f}(x)$
$\large \hat{f}(x) = \sum_{i = 0}^M \hat{f_i}(x) $
4. Conquer kaggle and world by trained model (make predictions, you can handle it by yorself)


### Step-by-step example of how GBM works

Let's try on a toy example to understand how GBM works. With this example we will restore a noisy function $\large y = cos(x) + \epsilon, \epsilon \sim \mathcal{N}(0, \frac{1}{5}), x \in [-5,5]$.

<img src='https://habrastorage.org/web/9fe/04d/7ba/9fe04d7ba5a645d49fc6aa3e875c8c41.jpg'   align='center'>

It's a regression problem that has target variable as a real value, that is why we will use the mean squared error loss function. We will generate 300 pairs of observations, and we will approximate with decision trees of depth 2. Let's put together everything we need to use GBM:
- Toy data $\large \left\{ (x_i, y_i) \right\}_{i=1, \ldots,300}$ ✓
- Number of iterations $\large M = 3$ ✓;
- The mean squared error loss function $\large L(y, f) = (y-f)^2$ ✓
Gradient $\large L(y, f) = L_2$ loss is just leavings of $\large r = (y - f)$ ✓;
- Decision trees as base algorithms $\large h(x)$ ✓;
- Hyperparameters of the decision trees: trees depth is equal to 2 ✓;

Initialization $\large \gamma$ and coefficients $\large \rho_t$ are simple with the mean squared error. Namely, we initialize GBM with the average value $\large \gamma = \frac{1}{n} \cdot \sum_{i = 1}^n y_i$, and all $\large \rho_t$ are equal to 1.

We will run GBM and draw two types of graphs: current approximation $\large \hat{f}(x)$ (blue graph), and also every tree $\large \hat{f_t}(x)$ built on their pseudo-residuals (green graph). The graph's number corresponds to the iteration's number:

<img src='https://habrastorage.org/web/edb/328/98a/edb32898ad014d8d95782759d11f63fb.png'   align='center'>

Note that by the second iteration our trees repeated the basic form of the function. However, at the first iteration we see that the algorithm built only the "left branch" of the function ($\large x \in [-5, -4]$). It happend due to the fact that our trees simply did not have enough depth to build a symmetrical branch at once, and the error on the left brach was bigger. So the right branch appeared only on the second iteration.

The rest of the process went as we expected: on every step our pseudo-residuals decresed, as a results, GBM better and better with each iteration approximated the original function. However, trees by construction cannot approximate a continuous function, therefore on this example GBM is useful, but not ideal. To play for yourself how GBM approximates functions, in the blog [Brilliantly wrong](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html) there are awesome interactive demo:

<img src='https://habrastorage.org/web/779/3e0/e66/7793e0e66b7d4871b6391a94cd5d4cf2.jpg'   align='center'>[http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html)


### Boosting parameters
Tunning parameters is very important stage, we can signifincantly improve our baseline model. It is very difficult to get answers to practical questions like – Which set of parameters should be tuned ? Let's take a look on general GBm parameters

All GBM parameters can be divided in three main groups:
1. Tree-Specific Parameters;
2. Boosting Parameters;
3. Miscellaneous Parameters. 


The optimal parameters of a model can depend on many scenarios. So it is impossible to create a comprehensive guide for doing so. 

# Xgboost

Its name stands for eXtreme Gradient Boosting, it was developed by Tianqi Chen and now is part of a wider collection of open-source libraries developed by the Distributed Machine Learning Community (DMLC). XGBoost is a scalable and accurate implementation of gradient boosting machines and it has proven to push the limits of computing power for boosted trees algorithms as it was built and developed for the sole purpose of model performance and computational speed. Specifically, it was engineered to exploit every bit of memory and hardware resources for tree boosting algorithms.

The implementation of XGBoost offers several advanced features for model tuning, computing environments and algorithm enhancement. It is capable of performing the three main forms of gradient boosting (Gradient Boosting (GB), Stochastic GB and Regularized GB) and it is robust enough to support fine tuning and addition of regularization parameters. According to Tianqi Chen, the latter is what makes it superior and different to other libraries. 

### Objective function

$$ \large obj^{(t)}=\sum_{i=1}^{n}[l(y_i,ŷ_{i}^{(t−1)})+g_{i}f_{t}(x_i)+\frac{1}{2}h_{i}f_{t}^{2}(x_{i})]+\Omega(f_t)+constant $$

where $\large g_{i}$ and $\large h_{i}$ – are first and second derivatives.
 
### Model Complexity

$$ \large \Omega(f) = \gamma T + \frac{1}{2}\lambda \sum_{j=1}^{T}(w_j^2)$$

where $\large w$ – vector of scores on leaves; $\large T$is the number of leaves; $\large \gamma$ and $\large \lambda$ – regularization parameters.


### Boosting settings

**Explanation after lightgbm theory**
`tree_method` – 'exact' (if you have time, you can try), 'approx', 'hist'(the best choise usually) <br>
`grow_policy` – 'depthwise', 'lossguide' (usually better)
`objective` – default is quite good (sometimes "count:poisson")


### XGBoost parameters tuning

Usually we start tuning parameters with these first: <br>
`n_estimators` - number of boosting rounds, better to use `early_stopping_rounds` <br>
`eta` – learning rate, increasing lr reduces convergence time. (usually default value works good)

### Control Overfitting
When you observe high training accuracy, but low tests accuracy, it is likely that you encounter overfitting problem.

There are in general two ways that you can control overfitting:

* The first way is to directly control model complexity <p>
`max_depth` - maximum depth of a tree, increase this value will make the model more complex; <br>
`gamma` - minimum loss reduction required to make a further partition on a leaf node of the tree.<br>
`min_child_weight` – minimum sum of instance weight (hessian) needed in a child.


* The second way is to add randomness to make training robust to noise <p>
`subsample` - subsample ratio of the training instance, <br>
`colsample_bytree` - subsample ratio of columns when constructing each tree. <br>


### Handle Imbalanced Dataset
There are two ways to improve it:

* If you care only about the ranking order (AUC) of your prediction
Balance the positive and negative weights, via `scale_pos_weight`
Use AUC for evaluation
* If you care about predicting the right probability
In such a case, you cannot re-balance the dataset
In such a case, set parameter `max_delta_step` to a finite number (say 1) will help convergence <br>

More about xgboost parameters: https://github.com/dmlc/xgboost/blob/master/doc/parameter.md

Always use `early_stopping_round` and tune `n_estimators` on validation.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

data = pd.read_csv('../data/credit_scoring_sample.csv', sep=";")

y = data["SeriousDlqin2yrs"].astype(int)
X = data.drop(["SeriousDlqin2yrs"], axis=1)
X = X.fillna(X.median())

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [2]:
y.value_counts()

0    35037
1    10026
Name: SeriousDlqin2yrs, dtype: int64

In [3]:
import xgboost as xgb


parameters = {
    #default
    'objective': 'reg:logistic',
    'eta': 0.1,
    'silent': 1,
    "nthread": 4,
    "random_seed": 1,
    "eval_metric": 'auc'
}


xgb_train = xgb.DMatrix(X, y, feature_names=X.columns)

In [4]:
results = xgb.cv(parameters, xgb_train, num_boost_round=100,
                 folds=skf, verbose_eval=1)

[0]	train-auc:0.827914+0.00201216	test-auc:0.820562+0.00556193
[1]	train-auc:0.830105+0.00201812	test-auc:0.822462+0.00547281
[2]	train-auc:0.83086+0.00164274	test-auc:0.8232+0.00566926
[3]	train-auc:0.832118+0.00112583	test-auc:0.824651+0.00587268
[4]	train-auc:0.833083+0.00150007	test-auc:0.825795+0.00534126
[5]	train-auc:0.833443+0.00184838	test-auc:0.826012+0.00522991
[6]	train-auc:0.833824+0.00183044	test-auc:0.826149+0.00511729
[7]	train-auc:0.834191+0.00162612	test-auc:0.826438+0.00530358
[8]	train-auc:0.834713+0.00189908	test-auc:0.826731+0.00506948
[9]	train-auc:0.834995+0.00159072	test-auc:0.826882+0.00524102
[10]	train-auc:0.835453+0.00171023	test-auc:0.827174+0.00513221
[11]	train-auc:0.835815+0.00160908	test-auc:0.827527+0.00539784
[12]	train-auc:0.836566+0.00147532	test-auc:0.828066+0.00522771
[13]	train-auc:0.836773+0.00141955	test-auc:0.82831+0.00521887
[14]	train-auc:0.837229+0.00161563	test-auc:0.828783+0.00503934
[15]	train-auc:0.837707+0.00152577	test-auc:0.829109+0

In [5]:
# added early stopping
num_rounds = 10000
results = xgb.cv(parameters, xgb_train, num_rounds, early_stopping_rounds=10,
                 folds=skf, verbose_eval=1)

[0]	train-auc:0.827914+0.00201216	test-auc:0.820562+0.00556193
[1]	train-auc:0.830105+0.00201812	test-auc:0.822462+0.00547281
[2]	train-auc:0.83086+0.00164274	test-auc:0.8232+0.00566926
[3]	train-auc:0.832118+0.00112583	test-auc:0.824651+0.00587268
[4]	train-auc:0.833083+0.00150007	test-auc:0.825795+0.00534126
[5]	train-auc:0.833443+0.00184838	test-auc:0.826012+0.00522991
[6]	train-auc:0.833824+0.00183044	test-auc:0.826149+0.00511729
[7]	train-auc:0.834191+0.00162612	test-auc:0.826438+0.00530358
[8]	train-auc:0.834713+0.00189908	test-auc:0.826731+0.00506948
[9]	train-auc:0.834995+0.00159072	test-auc:0.826882+0.00524102
[10]	train-auc:0.835453+0.00171023	test-auc:0.827174+0.00513221
[11]	train-auc:0.835815+0.00160908	test-auc:0.827527+0.00539784
[12]	train-auc:0.836566+0.00147532	test-auc:0.828066+0.00522771
[13]	train-auc:0.836773+0.00141955	test-auc:0.82831+0.00521887
[14]	train-auc:0.837229+0.00161563	test-auc:0.828783+0.00503934
[15]	train-auc:0.837707+0.00152577	test-auc:0.829109+0

In [6]:
%%time
parameters = {
    #default
    'objective': 'reg:logistic',
    'eta': 0.1,
    'silent': 1,
    "nthread": 4,
    "random_seed": 1,
    "eval_metric": 'auc',
    
    # regularization parameters
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7
}

results = xgb.cv(parameters, xgb_train, num_rounds, early_stopping_rounds=10,
                 folds=skf, verbose_eval=1)

[0]	train-auc:0.764574+0.00221952	test-auc:0.76103+0.00152838
[1]	train-auc:0.803184+0.00836168	test-auc:0.795713+0.00516933
[2]	train-auc:0.829761+0.00247429	test-auc:0.824885+0.00348889
[3]	train-auc:0.828645+0.0032039	test-auc:0.823018+0.00314941
[4]	train-auc:0.833933+0.00248571	test-auc:0.828286+0.00486735
[5]	train-auc:0.833894+0.00253292	test-auc:0.828691+0.00472594
[6]	train-auc:0.834414+0.00111002	test-auc:0.828737+0.00616273
[7]	train-auc:0.835575+0.00135683	test-auc:0.829559+0.00622821
[8]	train-auc:0.83629+0.00181739	test-auc:0.8305+0.00526671
[9]	train-auc:0.836568+0.00209568	test-auc:0.830395+0.00524195
[10]	train-auc:0.83717+0.00260457	test-auc:0.83116+0.00438677
[11]	train-auc:0.837208+0.00271824	test-auc:0.831201+0.0044317
[12]	train-auc:0.83787+0.00230315	test-auc:0.831694+0.00437275
[13]	train-auc:0.83809+0.00198743	test-auc:0.831616+0.00463882
[14]	train-auc:0.838418+0.00167979	test-auc:0.831417+0.00482319
[15]	train-auc:0.838272+0.00190361	test-auc:0.831428+0.00454

### We will be back after LightGBM theory

In [7]:
%%time
parameters = {
    #default
    'objective': 'reg:logistic',
    'eta': 0.1,
    'silent': 1,
    "nthread": 4,
    "random_seed": 1,
    "eval_metric": 'auc',
    
    # regularization parameters
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    
    #lightgbm approach
    'tree_method': 'hist'
}

results = xgb.cv(parameters, xgb_train, num_rounds, early_stopping_rounds=10,
                 folds=skf, verbose_eval=1)

[0]	train-auc:0.763774+0.00235156	test-auc:0.760565+0.00118614
[1]	train-auc:0.803024+0.00826439	test-auc:0.79589+0.00544105
[2]	train-auc:0.829803+0.00238068	test-auc:0.825031+0.00317944
[3]	train-auc:0.828453+0.00324755	test-auc:0.822698+0.00266877
[4]	train-auc:0.83358+0.00241821	test-auc:0.827807+0.00427303
[5]	train-auc:0.833488+0.00241429	test-auc:0.828046+0.00430865
[6]	train-auc:0.833875+0.00106693	test-auc:0.828117+0.00576497
[7]	train-auc:0.835276+0.0012984	test-auc:0.829296+0.005693
[8]	train-auc:0.836144+0.00176054	test-auc:0.830477+0.00484287
[9]	train-auc:0.836319+0.00197414	test-auc:0.830289+0.00493226
[10]	train-auc:0.83682+0.00249786	test-auc:0.831072+0.00403802
[11]	train-auc:0.836832+0.00266281	test-auc:0.831077+0.00405953
[12]	train-auc:0.837425+0.00240543	test-auc:0.831544+0.00400843
[13]	train-auc:0.837534+0.00201073	test-auc:0.831477+0.00446473
[14]	train-auc:0.837735+0.00190896	test-auc:0.831278+0.00464572
[15]	train-auc:0.83761+0.0022267	test-auc:0.831243+0.004

In [8]:
parameters = {
    #default
    'objective': 'reg:logistic',
    'eta': 0.1,
    'silent': 1,
    "nthread": 4,
    "random_seed": 1,
    "eval_metric": 'auc',
    
    # regularization parameters
    'max_leaves': 32,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    
    #lightgbm approach
    'tree_method': 'hist',
    'grow_policy': 'lossguide'
}

results = xgb.cv(parameters, xgb_train, num_rounds, early_stopping_rounds=10,
                 folds=skf, verbose_eval=1)

[0]	train-auc:0.7647+0.00296158	test-auc:0.760335+0.000753114
[1]	train-auc:0.804195+0.00763782	test-auc:0.79601+0.0057804
[2]	train-auc:0.831139+0.00318164	test-auc:0.825521+0.00188672
[3]	train-auc:0.829944+0.00395421	test-auc:0.823205+0.00176049
[4]	train-auc:0.835124+0.00279692	test-auc:0.828503+0.00431966
[5]	train-auc:0.835216+0.00242377	test-auc:0.829101+0.00446577
[6]	train-auc:0.835631+0.00134	test-auc:0.828914+0.00579192
[7]	train-auc:0.836792+0.00167504	test-auc:0.829778+0.00588304
[8]	train-auc:0.837714+0.00216174	test-auc:0.830637+0.00504056
[9]	train-auc:0.837885+0.00258239	test-auc:0.830633+0.00487559
[10]	train-auc:0.838466+0.00291253	test-auc:0.831299+0.00416135
[11]	train-auc:0.83848+0.00296868	test-auc:0.831162+0.0043099
[12]	train-auc:0.839157+0.00263938	test-auc:0.831767+0.00421208
[13]	train-auc:0.839335+0.00218933	test-auc:0.831686+0.00467014
[14]	train-auc:0.839724+0.002151	test-auc:0.831713+0.00485974
[15]	train-auc:0.839556+0.00238155	test-auc:0.831685+0.00464

In [9]:
parameters = {
    #default
    'objective': 'reg:logistic',
    'eta': 0.01, #reduced to get higher score
    'silent': 1,
    "nthread": 4,
    "random_seed": 1,
    "eval_metric": 'auc',
    
    # regularization parameters
    'max_leaves': 32,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    
    #lightgbm approach
    'tree_method': 'hist',
    'grow_policy': 'lossguide'
}

results = xgb.cv(parameters, xgb_train, num_rounds, early_stopping_rounds=30,
                 folds=skf, verbose_eval=1)

[0]	train-auc:0.7647+0.00296158	test-auc:0.760335+0.000753114
[1]	train-auc:0.803423+0.00710127	test-auc:0.795271+0.0055919
[2]	train-auc:0.829531+0.00296606	test-auc:0.823744+0.00190007
[3]	train-auc:0.82772+0.00394377	test-auc:0.821172+0.00172675
[4]	train-auc:0.83357+0.00264106	test-auc:0.827151+0.00440435
[5]	train-auc:0.833575+0.00226768	test-auc:0.827752+0.00451364
[6]	train-auc:0.833626+0.000543748	test-auc:0.827643+0.00621962
[7]	train-auc:0.834516+0.00112372	test-auc:0.82863+0.00655088
[8]	train-auc:0.835188+0.00127673	test-auc:0.829599+0.00550326
[9]	train-auc:0.835094+0.00186442	test-auc:0.829389+0.00543588
[10]	train-auc:0.835439+0.00213486	test-auc:0.829893+0.0046465
[11]	train-auc:0.83524+0.00204886	test-auc:0.829797+0.00476401
[12]	train-auc:0.835946+0.00178645	test-auc:0.830338+0.00462957
[13]	train-auc:0.835779+0.00122571	test-auc:0.82999+0.00522123
[14]	train-auc:0.835591+0.00111346	test-auc:0.829615+0.00547603
[15]	train-auc:0.835259+0.00149565	test-auc:0.829512+0.00

[129]	train-auc:0.842255+0.00200607	test-auc:0.834867+0.00417024
[130]	train-auc:0.842331+0.00205972	test-auc:0.834906+0.00413769
[131]	train-auc:0.842298+0.00205711	test-auc:0.834897+0.00414591
[132]	train-auc:0.842287+0.00203921	test-auc:0.834891+0.00415642
[133]	train-auc:0.842306+0.00205486	test-auc:0.8349+0.00416534
[134]	train-auc:0.842382+0.00204827	test-auc:0.834945+0.00414652
[135]	train-auc:0.842393+0.00206267	test-auc:0.834931+0.00415955
[136]	train-auc:0.842393+0.00208045	test-auc:0.83491+0.0041603
[137]	train-auc:0.84243+0.00200485	test-auc:0.834898+0.00419081
[138]	train-auc:0.842417+0.0020194	test-auc:0.834866+0.00420002
[139]	train-auc:0.84241+0.00199417	test-auc:0.834845+0.00421419
[140]	train-auc:0.842412+0.00201706	test-auc:0.834873+0.00420395
[141]	train-auc:0.84243+0.00206585	test-auc:0.834872+0.00419913
[142]	train-auc:0.842427+0.00209165	test-auc:0.834886+0.00418445
[143]	train-auc:0.84249+0.00211392	test-auc:0.834962+0.00418106
[144]	train-auc:0.842524+0.0020937

[257]	train-auc:0.844992+0.00166045	test-auc:0.835607+0.00418467
[258]	train-auc:0.845+0.00165784	test-auc:0.8356+0.0041918
[259]	train-auc:0.845008+0.00165618	test-auc:0.835597+0.00417863
[260]	train-auc:0.845031+0.00165821	test-auc:0.835613+0.00417446
[261]	train-auc:0.84505+0.00164078	test-auc:0.8356+0.00419035
[262]	train-auc:0.845062+0.00164509	test-auc:0.835611+0.00418959
[263]	train-auc:0.845083+0.00164513	test-auc:0.835623+0.00419077
[264]	train-auc:0.845112+0.00166441	test-auc:0.835652+0.0041736
[265]	train-auc:0.845162+0.00165215	test-auc:0.835669+0.00418881
[266]	train-auc:0.845207+0.00168211	test-auc:0.835692+0.00417742
[267]	train-auc:0.845216+0.00167676	test-auc:0.83568+0.00418523
[268]	train-auc:0.845252+0.00166816	test-auc:0.835689+0.00418008
[269]	train-auc:0.845273+0.00166666	test-auc:0.83569+0.00418264
[270]	train-auc:0.845285+0.00167513	test-auc:0.835699+0.0041878
[271]	train-auc:0.84532+0.00167595	test-auc:0.835703+0.0041816
[272]	train-auc:0.845357+0.00167384	test

[385]	train-auc:0.847843+0.00155252	test-auc:0.836183+0.00404312
[386]	train-auc:0.847858+0.00155136	test-auc:0.836185+0.00405
[387]	train-auc:0.847869+0.00155696	test-auc:0.836184+0.00405536
[388]	train-auc:0.847886+0.00155031	test-auc:0.836187+0.0040589
[389]	train-auc:0.847918+0.00154355	test-auc:0.836187+0.00405301
[390]	train-auc:0.847942+0.00153755	test-auc:0.836197+0.00404866
[391]	train-auc:0.847973+0.00152413	test-auc:0.836193+0.00404472
[392]	train-auc:0.847987+0.00152566	test-auc:0.836186+0.00404987
[393]	train-auc:0.848022+0.00154851	test-auc:0.836208+0.00402917
[394]	train-auc:0.848049+0.00154433	test-auc:0.836213+0.004026
[395]	train-auc:0.848066+0.00153943	test-auc:0.836222+0.00402384
[396]	train-auc:0.848103+0.00153304	test-auc:0.836222+0.00402348
[397]	train-auc:0.848129+0.00152936	test-auc:0.836222+0.00402493
[398]	train-auc:0.848148+0.00153672	test-auc:0.836231+0.00401977
[399]	train-auc:0.848169+0.00155189	test-auc:0.836238+0.00401661
[400]	train-auc:0.848206+0.0015

[513]	train-auc:0.850592+0.00153583	test-auc:0.836505+0.00394353
[514]	train-auc:0.850628+0.00153752	test-auc:0.836505+0.00393785
[515]	train-auc:0.850654+0.00152639	test-auc:0.836508+0.00393844
[516]	train-auc:0.85067+0.00152459	test-auc:0.836498+0.00392874
[517]	train-auc:0.85068+0.0015201	test-auc:0.836497+0.00392565
[518]	train-auc:0.850706+0.00151113	test-auc:0.836494+0.00392959
[519]	train-auc:0.850726+0.00152173	test-auc:0.836506+0.0039214
[520]	train-auc:0.850758+0.00152304	test-auc:0.83651+0.00390828
[521]	train-auc:0.850787+0.00150779	test-auc:0.836519+0.00390933
[522]	train-auc:0.850803+0.00151626	test-auc:0.836518+0.00390718
[523]	train-auc:0.850826+0.00150125	test-auc:0.836512+0.00390527
[524]	train-auc:0.850836+0.00150124	test-auc:0.836507+0.00390619
[525]	train-auc:0.850869+0.00148762	test-auc:0.836504+0.00391049
[526]	train-auc:0.850902+0.00148407	test-auc:0.836511+0.00391777
[527]	train-auc:0.850929+0.00149371	test-auc:0.836516+0.00391909
[528]	train-auc:0.85095+0.0015

### Some useful notes

1. Custom objective and evaludation functions – https://github.com/dmlc/xgboost/blob/master/demo/guide-python/custom_objective.py
2. Never use timeseries validation with **xgb.cv**, it is broken :( 
3. Ivestigate your task and metrics. There are many objective functions that are worth to try.


# LightGBM

LightGBM is the new GBDT algorithm with **GOSS** and **EFB**. 

### GOSS

**Gradient-based One-Side Sampling (GOSS).** While there is no native weight for data instance in GBDT, they notice that data instances with different gradients play different roles in the computation of information gain. In particular, according to the definition of information gain, those instances with larger gradients (i.e., under-trained instances) will contribute more to the information gain. Therefore, when down sampling the data instances, in order to retain the accuracy of information gain estimation, we should better keep those instances with large gradients (e.g., larger than a pre-defined threshold, or among the top percentiles), and only randomly drop those instances with small gradients. We prove that such a treatment can lead to a more accurate gain estimation than uniformly random sampling, with the same target sampling rate, especially when the value of information gain has a large range.

### EFB

**Exclusive Feature Bundling (EFB).** Usually in real applications, although there are a large number of features, the feature space is quite sparse, which provides us a possibility of designing a nearly lossless approach to reduce the number of effective features. Specifically, in a sparse feature space, many features are (almost) exclusive, i.e., they rarely take nonzero values simultaneously. Examples include the one-hot features (e.g., one-hot word representation in text mining). We can safely bundle such exclusive features. To this end, they design an efficient algorithm by reducing the optimal bundling problem to a graph coloring problem (by taking features as vertices and adding edges for every two features if they are not mutually exclusive), and solving it by a greedy algorithm with a constant approximation ratio.

LightGBM uses the histogram based algorithms, which bucketing continuous feature(attribute) values into discrete bins, to speed up training procedure and reduce memory usage. In 2017 it was inplemented in Xgboost too. 

### Tree Spliting

**Leaf-wise (Best-first) Tree Growth**

Most decision tree learning algorithms grow tree by level (depth)-wise, like the following image:

<img src='https://github.com/Microsoft/LightGBM/blob/master/docs/_static/images/level-wise.png?raw=true'   align='center'>

LightGBM grows tree by leaf-wise (best-first). It will choose the leaf with max delta loss to grow. When growing same #leaf, leaf-wise algorithm can reduce more loss than level-wise algorithm.

Leaf-wise may cause over-fitting when #data is small. So, LightGBM can use an additional parameter max_depth to limit depth of tree and avoid over-fitting (tree still grows by leaf-wise).

<img src='https://github.com/Microsoft/LightGBM/raw/master/docs/_static/images/leaf-wise.png'   align='center'>

### Histogram based

LightGBM uses the histogram based algorithms, which bucketing continuous feature(attribute) values into discrete bins, to speed up training procedure and reduce memory usage. 

### Working with Categorical Features

We often convert the categorical features into one-hot coding. However, it is not a good solution in tree learner. The reason is, for the high cardinality categorical features, it will grow the very unbalance tree, and needs to grow very deep to achieve the good accuracy.

The basic idea is reordering the categories according to the relevance of training target. More specifically, reordering the histogram (of categorical feature) according to it's accumulate values (`sum_gradient` / `sum_hessian`), then find the best split on the sorted histogram.

On practice, you need to try both methods and choose the best for your data.


### Lightgbm Parameters Tunning


**Core parameters**

Begin with `learning_rate` (shrinkage rate) parameter setting with default value <br>
`n_estimators`(number of boosting iterations) set 10k (some big number).<br>
`num_leaves` (number of leaves in one tree) set 155 as a starting point.<br>
`application` –  the same as for xgboost `objective` (adding "regression_l1")

**Control Overfitting and Accuracy**

`max_depth` – max tree depth (`default=-1`). Start with tunning `num_leaves`, in the end try to change `max_depth`; <br>
`min_data_in_leaf` – very important parameter that helps to control overfitting (`default=20`);<br>
`colsample_bytree` – select part of features on each iteration (`default=1.0`). Always have to be tunned! <br>
`subsample` – select part of data without resampling (`default=1.0`). To enable it, set `subsample_freq` = 1 (other values always work worse)
`early_stopping_round` – as in xgboost



In [10]:
import lightgbm as lgb

parameters = {
    'objective': 'binary',
    'learning_rate': 0.1,
    'num_threads': 4,
    "metric": 'auc'
}
n_rounds = 10000

lgb_train = lgb.Dataset(X, label=y, free_raw_data=False)

In [11]:
result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf.split(X, y), early_stopping_rounds=10, verbose_eval=1, )

[1]	cv_agg's auc: 0.821977 + 0.00222991
[2]	cv_agg's auc: 0.823861 + 0.00328635
[3]	cv_agg's auc: 0.82527 + 0.00332871
[4]	cv_agg's auc: 0.826186 + 0.00296335
[5]	cv_agg's auc: 0.826827 + 0.00241723
[6]	cv_agg's auc: 0.826959 + 0.00256874
[7]	cv_agg's auc: 0.827595 + 0.00212654
[8]	cv_agg's auc: 0.827977 + 0.00288587
[9]	cv_agg's auc: 0.828348 + 0.00270048
[10]	cv_agg's auc: 0.829124 + 0.00314304
[11]	cv_agg's auc: 0.829921 + 0.00348139
[12]	cv_agg's auc: 0.830513 + 0.00341132
[13]	cv_agg's auc: 0.830975 + 0.00330805
[14]	cv_agg's auc: 0.831296 + 0.00313771
[15]	cv_agg's auc: 0.831941 + 0.00254119
[16]	cv_agg's auc: 0.832095 + 0.00264504
[17]	cv_agg's auc: 0.832354 + 0.00241476
[18]	cv_agg's auc: 0.832625 + 0.00235101
[19]	cv_agg's auc: 0.832887 + 0.00235174
[20]	cv_agg's auc: 0.83282 + 0.00229104
[21]	cv_agg's auc: 0.833198 + 0.00244485
[22]	cv_agg's auc: 0.833427 + 0.00244739
[23]	cv_agg's auc: 0.833615 + 0.00243255
[24]	cv_agg's auc: 0.833838 + 0.0023752
[25]	cv_agg's auc: 0.834076 

In [12]:
from sklearn.metrics import roc_auc_score
def check_train_score(params, lgb_data, data, target, kf, num_rounds):
    roc_auc = []
    for train, val in kf.split(data,target):
        temp_lgb_train = lgb_data.subset(train)
        temp_lgb_val = lgb_data.subset(val)
        temp_model = lgb.train(params, temp_lgb_train, num_rounds, verbose_eval=num_rounds)
        roc_auc.append([roc_auc_score(target[train],temp_model.predict(data.loc[train])), roc_auc_score(target[val],temp_model.predict(data.loc[val]))])
    return np.mean(roc_auc, axis=0)

check_train_score(parameters, lgb_train, X, y, skf, 43)

array([0.85309676, 0.83568191])

In [13]:
parameters = {
    #default
    'objective': 'binary',
    'learning_rate': 0.1,
    'num_threads': 4,
    "metric": 'auc',
    
    #regularization
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 1
}

result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf.split(X, y), early_stopping_rounds=10, verbose_eval=1, )

[1]	cv_agg's auc: 0.757568 + 0.00204557
[2]	cv_agg's auc: 0.813124 + 0.00236009
[3]	cv_agg's auc: 0.827831 + 0.00248928
[4]	cv_agg's auc: 0.825591 + 0.00234024
[5]	cv_agg's auc: 0.824035 + 0.00232089
[6]	cv_agg's auc: 0.830438 + 0.00241013
[7]	cv_agg's auc: 0.833095 + 0.00207274
[8]	cv_agg's auc: 0.833152 + 0.0025069
[9]	cv_agg's auc: 0.833854 + 0.00251311
[10]	cv_agg's auc: 0.834122 + 0.00229212
[11]	cv_agg's auc: 0.834377 + 0.00237226
[12]	cv_agg's auc: 0.834079 + 0.00222016
[13]	cv_agg's auc: 0.833967 + 0.00211508
[14]	cv_agg's auc: 0.834137 + 0.00218341
[15]	cv_agg's auc: 0.834195 + 0.0021822
[16]	cv_agg's auc: 0.834299 + 0.00206047
[17]	cv_agg's auc: 0.834155 + 0.00199677
[18]	cv_agg's auc: 0.834763 + 0.00193014
[19]	cv_agg's auc: 0.835258 + 0.00206007
[20]	cv_agg's auc: 0.835435 + 0.00203493
[21]	cv_agg's auc: 0.835423 + 0.00209545
[22]	cv_agg's auc: 0.835337 + 0.00207262
[23]	cv_agg's auc: 0.83548 + 0.00204717
[24]	cv_agg's auc: 0.835723 + 0.00216287
[25]	cv_agg's auc: 0.835556 

In [14]:
parameters = {
    #default
    'objective': 'binary',
    'learning_rate': 0.1,
    'num_threads': 4,
    "metric": 'auc',
    
    #regularization
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 15
}

result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf.split(X, y), early_stopping_rounds=10, verbose_eval=1, )

[1]	cv_agg's auc: 0.75738 + 0.00190223
[2]	cv_agg's auc: 0.813216 + 0.00267186
[3]	cv_agg's auc: 0.827903 + 0.00263115
[4]	cv_agg's auc: 0.825654 + 0.00241034
[5]	cv_agg's auc: 0.824041 + 0.0023371
[6]	cv_agg's auc: 0.830501 + 0.00240964
[7]	cv_agg's auc: 0.833034 + 0.00204312
[8]	cv_agg's auc: 0.833035 + 0.00239431
[9]	cv_agg's auc: 0.833762 + 0.00244048
[10]	cv_agg's auc: 0.834036 + 0.00232882
[11]	cv_agg's auc: 0.834336 + 0.00227354
[12]	cv_agg's auc: 0.834011 + 0.0021514
[13]	cv_agg's auc: 0.834028 + 0.00212603
[14]	cv_agg's auc: 0.834263 + 0.00216131
[15]	cv_agg's auc: 0.834253 + 0.00209065
[16]	cv_agg's auc: 0.834361 + 0.00199839
[17]	cv_agg's auc: 0.834239 + 0.00203559
[18]	cv_agg's auc: 0.834856 + 0.00189129
[19]	cv_agg's auc: 0.835327 + 0.001999
[20]	cv_agg's auc: 0.835523 + 0.00201681
[21]	cv_agg's auc: 0.835529 + 0.0020554
[22]	cv_agg's auc: 0.83544 + 0.00210205
[23]	cv_agg's auc: 0.835498 + 0.00200818
[24]	cv_agg's auc: 0.835751 + 0.00204492
[25]	cv_agg's auc: 0.835586 + 0.

In [15]:
parameters = {
    #default
    'objective': 'binary',
    'learning_rate': 0.01,
    'num_threads': 4,
    "metric": 'auc',
    
    #regularization
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 15
}


result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf.split(X, y), early_stopping_rounds=50, verbose_eval=1, )

[1]	cv_agg's auc: 0.75738 + 0.00190223
[2]	cv_agg's auc: 0.812425 + 0.00234425
[3]	cv_agg's auc: 0.827285 + 0.00238678
[4]	cv_agg's auc: 0.824187 + 0.00222083
[5]	cv_agg's auc: 0.822935 + 0.00243823
[6]	cv_agg's auc: 0.829251 + 0.00252414
[7]	cv_agg's auc: 0.832113 + 0.0019764
[8]	cv_agg's auc: 0.831922 + 0.00248599
[9]	cv_agg's auc: 0.832753 + 0.00237166
[10]	cv_agg's auc: 0.833204 + 0.00222853
[11]	cv_agg's auc: 0.833527 + 0.0021929
[12]	cv_agg's auc: 0.83312 + 0.00211504
[13]	cv_agg's auc: 0.832657 + 0.00204377
[14]	cv_agg's auc: 0.832061 + 0.00198255
[15]	cv_agg's auc: 0.832105 + 0.00211448
[16]	cv_agg's auc: 0.832161 + 0.00217127
[17]	cv_agg's auc: 0.831476 + 0.00213298
[18]	cv_agg's auc: 0.832586 + 0.00198784
[19]	cv_agg's auc: 0.833418 + 0.0020231
[20]	cv_agg's auc: 0.833951 + 0.00196813
[21]	cv_agg's auc: 0.833874 + 0.00204159
[22]	cv_agg's auc: 0.833771 + 0.00206252
[23]	cv_agg's auc: 0.833401 + 0.00208491
[24]	cv_agg's auc: 0.833785 + 0.00209862
[25]	cv_agg's auc: 0.833542 + 

[201]	cv_agg's auc: 0.835891 + 0.00254702
[202]	cv_agg's auc: 0.835904 + 0.00254588
[203]	cv_agg's auc: 0.835917 + 0.00254627
[204]	cv_agg's auc: 0.835921 + 0.00254042
[205]	cv_agg's auc: 0.835932 + 0.00253368
[206]	cv_agg's auc: 0.835952 + 0.00255244
[207]	cv_agg's auc: 0.835941 + 0.00253889
[208]	cv_agg's auc: 0.83597 + 0.00253017
[209]	cv_agg's auc: 0.835967 + 0.00252926
[210]	cv_agg's auc: 0.835999 + 0.00252974
[211]	cv_agg's auc: 0.835993 + 0.00253455
[212]	cv_agg's auc: 0.836015 + 0.00254268
[213]	cv_agg's auc: 0.836018 + 0.00254455
[214]	cv_agg's auc: 0.836058 + 0.00253981
[215]	cv_agg's auc: 0.836053 + 0.00253169
[216]	cv_agg's auc: 0.83604 + 0.00252885
[217]	cv_agg's auc: 0.836033 + 0.0025234
[218]	cv_agg's auc: 0.83603 + 0.00252601
[219]	cv_agg's auc: 0.836074 + 0.0025243
[220]	cv_agg's auc: 0.836089 + 0.00253197
[221]	cv_agg's auc: 0.836105 + 0.0025158
[222]	cv_agg's auc: 0.836105 + 0.00252548
[223]	cv_agg's auc: 0.836097 + 0.00251519
[224]	cv_agg's auc: 0.83611 + 0.00252342

[404]	cv_agg's auc: 0.836619 + 0.00258069
[405]	cv_agg's auc: 0.83662 + 0.00258298
[406]	cv_agg's auc: 0.836622 + 0.00258063
[407]	cv_agg's auc: 0.836631 + 0.00257725
[408]	cv_agg's auc: 0.836625 + 0.00258094
[409]	cv_agg's auc: 0.836628 + 0.00257962
[410]	cv_agg's auc: 0.836633 + 0.00258171
[411]	cv_agg's auc: 0.836631 + 0.00258289
[412]	cv_agg's auc: 0.836629 + 0.00257703
[413]	cv_agg's auc: 0.836621 + 0.00257231
[414]	cv_agg's auc: 0.836623 + 0.00257476
[415]	cv_agg's auc: 0.836623 + 0.00256845
[416]	cv_agg's auc: 0.836628 + 0.00257139
[417]	cv_agg's auc: 0.836634 + 0.00258312
[418]	cv_agg's auc: 0.836648 + 0.00258034
[419]	cv_agg's auc: 0.836643 + 0.00258452
[420]	cv_agg's auc: 0.836639 + 0.00258451
[421]	cv_agg's auc: 0.836633 + 0.00257835
[422]	cv_agg's auc: 0.836635 + 0.00258374
[423]	cv_agg's auc: 0.836652 + 0.00259039
[424]	cv_agg's auc: 0.836651 + 0.00259925
[425]	cv_agg's auc: 0.83665 + 0.00260744
[426]	cv_agg's auc: 0.836648 + 0.00261115
[427]	cv_agg's auc: 0.836644 + 0.002

In [16]:
print(max(result['auc-mean']), len(result['auc-mean']))
print(check_train_score(parameters, lgb_train, X, y, skf, len(result['auc-mean'])))


0.8367307292331999 527
[0.85680227 0.83673073]


In [17]:
parameters = {
    #default
    'objective': 'binary',
    'learning_rate': 0.01,
    'num_threads': 4,
    "metric": 'auc',
    
    #regularization
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 50
}


result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf.split(X, y), early_stopping_rounds=50, verbose_eval=1, )

[1]	cv_agg's auc: 0.757935 + 0.00276078
[2]	cv_agg's auc: 0.812707 + 0.00178953
[3]	cv_agg's auc: 0.827557 + 0.00221063
[4]	cv_agg's auc: 0.824757 + 0.00185163
[5]	cv_agg's auc: 0.823352 + 0.00190702
[6]	cv_agg's auc: 0.829445 + 0.00224327
[7]	cv_agg's auc: 0.832228 + 0.00184373
[8]	cv_agg's auc: 0.832116 + 0.0022381
[9]	cv_agg's auc: 0.832928 + 0.00220667
[10]	cv_agg's auc: 0.833362 + 0.00203781
[11]	cv_agg's auc: 0.833572 + 0.00202378
[12]	cv_agg's auc: 0.833246 + 0.00194476
[13]	cv_agg's auc: 0.832798 + 0.00193023
[14]	cv_agg's auc: 0.832234 + 0.0019452
[15]	cv_agg's auc: 0.832267 + 0.00199931
[16]	cv_agg's auc: 0.832408 + 0.00201261
[17]	cv_agg's auc: 0.831798 + 0.00192847
[18]	cv_agg's auc: 0.832876 + 0.00181175
[19]	cv_agg's auc: 0.83359 + 0.00184104
[20]	cv_agg's auc: 0.834054 + 0.00179072
[21]	cv_agg's auc: 0.834021 + 0.00184837
[22]	cv_agg's auc: 0.833929 + 0.00187011
[23]	cv_agg's auc: 0.833576 + 0.00191087
[24]	cv_agg's auc: 0.834 + 0.00193414
[25]	cv_agg's auc: 0.833718 + 0

[201]	cv_agg's auc: 0.835938 + 0.0023489
[202]	cv_agg's auc: 0.835953 + 0.0023439
[203]	cv_agg's auc: 0.835963 + 0.00234984
[204]	cv_agg's auc: 0.835972 + 0.00234959
[205]	cv_agg's auc: 0.835968 + 0.00234242
[206]	cv_agg's auc: 0.835999 + 0.00235491
[207]	cv_agg's auc: 0.83598 + 0.00234154
[208]	cv_agg's auc: 0.836013 + 0.00233835
[209]	cv_agg's auc: 0.836017 + 0.00234146
[210]	cv_agg's auc: 0.836052 + 0.00235799
[211]	cv_agg's auc: 0.836049 + 0.00235744
[212]	cv_agg's auc: 0.836064 + 0.00237208
[213]	cv_agg's auc: 0.836061 + 0.00236655
[214]	cv_agg's auc: 0.836084 + 0.00237215
[215]	cv_agg's auc: 0.836081 + 0.00237076
[216]	cv_agg's auc: 0.836064 + 0.00236547
[217]	cv_agg's auc: 0.836062 + 0.00234895
[218]	cv_agg's auc: 0.836061 + 0.00235857
[219]	cv_agg's auc: 0.836098 + 0.00235366
[220]	cv_agg's auc: 0.836122 + 0.00236819
[221]	cv_agg's auc: 0.836136 + 0.00236131
[222]	cv_agg's auc: 0.836144 + 0.00236725
[223]	cv_agg's auc: 0.836138 + 0.00236208
[224]	cv_agg's auc: 0.836149 + 0.0023

[403]	cv_agg's auc: 0.836778 + 0.00242763
[404]	cv_agg's auc: 0.836777 + 0.00242706
[405]	cv_agg's auc: 0.836777 + 0.00242737
[406]	cv_agg's auc: 0.836782 + 0.00243112
[407]	cv_agg's auc: 0.836788 + 0.00242749
[408]	cv_agg's auc: 0.836776 + 0.00242285
[409]	cv_agg's auc: 0.83678 + 0.00241812
[410]	cv_agg's auc: 0.836791 + 0.00241917
[411]	cv_agg's auc: 0.836793 + 0.002419
[412]	cv_agg's auc: 0.836795 + 0.0024245
[413]	cv_agg's auc: 0.836788 + 0.0024238
[414]	cv_agg's auc: 0.836789 + 0.00243215
[415]	cv_agg's auc: 0.836799 + 0.00242507
[416]	cv_agg's auc: 0.836793 + 0.00242726
[417]	cv_agg's auc: 0.836795 + 0.0024353
[418]	cv_agg's auc: 0.836804 + 0.00243781
[419]	cv_agg's auc: 0.836798 + 0.00243766
[420]	cv_agg's auc: 0.83679 + 0.00244232
[421]	cv_agg's auc: 0.836782 + 0.00244534
[422]	cv_agg's auc: 0.836782 + 0.00244088
[423]	cv_agg's auc: 0.836789 + 0.00243308
[424]	cv_agg's auc: 0.836796 + 0.00244088
[425]	cv_agg's auc: 0.836791 + 0.00244473
[426]	cv_agg's auc: 0.836791 + 0.00245549

In [18]:
print(max(result['auc-mean']), len(result['auc-mean']))
print(check_train_score(parameters, lgb_train, X, y, skf, len(result['auc-mean'])))



0.8368799815573268 524
[0.85489541 0.83687998]


In [19]:
X.NumberOfDependents.value_counts()

0.0     26626
1.0      8106
2.0      6071
3.0      3004
4.0       951
5.0       226
6.0        57
7.0        14
8.0         6
10.0        1
9.0         1
Name: NumberOfDependents, dtype: int64

In [20]:
parameters = {
    #default
    'objective': 'binary',
    'learning_rate': 0.01,
    'num_threads': 4,
    "metric": 'auc',
    
    #regularization
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 15,
    
    #categorical features
    'cat_smooth': 10,
    'min_data_per_group': 50
}
lgb_train = lgb.Dataset(X, label=y, free_raw_data=False, categorical_feature=['NumberOfDependents'])
result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf.split(X, y), early_stopping_rounds=50, verbose_eval=1, 
               )

[1]	cv_agg's auc: 0.757937 + 0.00326715
[2]	cv_agg's auc: 0.812465 + 0.00238866
[3]	cv_agg's auc: 0.826023 + 0.00277399
[4]	cv_agg's auc: 0.82368 + 0.00305837


/usr/local/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[5]	cv_agg's auc: 0.821952 + 0.00327947
[6]	cv_agg's auc: 0.82811 + 0.00286699
[7]	cv_agg's auc: 0.83146 + 0.00243925
[8]	cv_agg's auc: 0.830996 + 0.00279731
[9]	cv_agg's auc: 0.832046 + 0.00267855
[10]	cv_agg's auc: 0.832393 + 0.00233694
[11]	cv_agg's auc: 0.832579 + 0.00233081
[12]	cv_agg's auc: 0.832181 + 0.00227486
[13]	cv_agg's auc: 0.831743 + 0.00217356
[14]	cv_agg's auc: 0.831147 + 0.0021768
[15]	cv_agg's auc: 0.831229 + 0.00225714
[16]	cv_agg's auc: 0.831324 + 0.0022737
[17]	cv_agg's auc: 0.830743 + 0.00233376
[18]	cv_agg's auc: 0.831842 + 0.00217317
[19]	cv_agg's auc: 0.832499 + 0.00225665
[20]	cv_agg's auc: 0.833059 + 0.00224372
[21]	cv_agg's auc: 0.83302 + 0.00228522
[22]	cv_agg's auc: 0.832881 + 0.00233827
[23]	cv_agg's auc: 0.832544 + 0.00238421
[24]	cv_agg's auc: 0.83294 + 0.00239642
[25]	cv_agg's auc: 0.832717 + 0.00239675
[26]	cv_agg's auc: 0.832558 + 0.00238491
[27]	cv_agg's auc: 0.832546 + 0.00241649
[28]	cv_agg's auc: 0.832727 + 0.00241844
[29]	cv_agg's auc: 0.832723

[205]	cv_agg's auc: 0.83556 + 0.00275297
[206]	cv_agg's auc: 0.835591 + 0.00275343
[207]	cv_agg's auc: 0.835588 + 0.00274078
[208]	cv_agg's auc: 0.835617 + 0.00271623
[209]	cv_agg's auc: 0.83561 + 0.00271963
[210]	cv_agg's auc: 0.835632 + 0.00272079
[211]	cv_agg's auc: 0.835644 + 0.00271537
[212]	cv_agg's auc: 0.835665 + 0.00272552
[213]	cv_agg's auc: 0.835665 + 0.00272189
[214]	cv_agg's auc: 0.835708 + 0.00272338
[215]	cv_agg's auc: 0.835708 + 0.00272387
[216]	cv_agg's auc: 0.835704 + 0.00271448
[217]	cv_agg's auc: 0.835689 + 0.0026993
[218]	cv_agg's auc: 0.835687 + 0.0027029
[219]	cv_agg's auc: 0.835723 + 0.0027015
[220]	cv_agg's auc: 0.835743 + 0.00270934
[221]	cv_agg's auc: 0.835763 + 0.0027003
[222]	cv_agg's auc: 0.835762 + 0.00270712
[223]	cv_agg's auc: 0.835762 + 0.00270093
[224]	cv_agg's auc: 0.835779 + 0.00270433
[225]	cv_agg's auc: 0.835791 + 0.00270552
[226]	cv_agg's auc: 0.835796 + 0.00269768
[227]	cv_agg's auc: 0.835821 + 0.00268822
[228]	cv_agg's auc: 0.835842 + 0.0026995

[406]	cv_agg's auc: 0.836496 + 0.00253675
[407]	cv_agg's auc: 0.836504 + 0.00253397
[408]	cv_agg's auc: 0.836503 + 0.00253909
[409]	cv_agg's auc: 0.836506 + 0.00253955
[410]	cv_agg's auc: 0.836507 + 0.002543
[411]	cv_agg's auc: 0.836505 + 0.00253421
[412]	cv_agg's auc: 0.836506 + 0.00253084
[413]	cv_agg's auc: 0.836497 + 0.00252677
[414]	cv_agg's auc: 0.836498 + 0.00252891
[415]	cv_agg's auc: 0.836496 + 0.00252201
[416]	cv_agg's auc: 0.836499 + 0.00251331
[417]	cv_agg's auc: 0.836504 + 0.00252146
[418]	cv_agg's auc: 0.836513 + 0.00252584
[419]	cv_agg's auc: 0.836504 + 0.00252559
[420]	cv_agg's auc: 0.836505 + 0.00252119
[421]	cv_agg's auc: 0.836501 + 0.00251878
[422]	cv_agg's auc: 0.836509 + 0.00252323
[423]	cv_agg's auc: 0.836525 + 0.00252936
[424]	cv_agg's auc: 0.836524 + 0.00253094
[425]	cv_agg's auc: 0.836523 + 0.00254118
[426]	cv_agg's auc: 0.836521 + 0.00253985
[427]	cv_agg's auc: 0.836513 + 0.0025361
[428]	cv_agg's auc: 0.836522 + 0.00253881
[429]	cv_agg's auc: 0.836524 + 0.0025

In [21]:
max(result['auc-mean']), len(result['auc-mean'])

(0.8366562162284218, 543)

# CatBoost

CatBoost is a recently open-sourced machine learning algorithm from Yandex. It can easily integrate with deep learning frameworks like Google’s TensorFlow and Apple’s Core ML. It can work with diverse data types to help solve a wide range of problems that businesses face today. To top it up, it provides best-in-class accuracy.

“CatBoost” name comes from two words “Category” and “Boosting”.

### Binarization

Before learning, the possible values of objects are divided into disjoint ranges ( buckets ) delimited by the threshold values ( splits ). The size of the binarization (the number of splits) is determined by the starting parameters. 

Binarization is also used to categorical features. A random subset of the dataset is used for this purpose on large datasets.

Types: Median, Uniform, UniformAndQuantiles (Median + Uniform), MaxLogSum (maximize $ \large \sum_{i=1}^{n}log(weight)$, $\large n $ - number of unique objects, $\large weight $ – the number of times an object in the bucket is repeated), MinEntropy (minimize $ \large \sum_{i=1}^{n}weight \times log(weight)$), GreedyLogSum(maximize greedy).


### Dealing with Categorical Features

The main difference between Xgboost/LightGBM vs CatBoost is transforming categorical features.

Before each split is selected in the tree, categorical features are transformed to numeric features. This is done using various statistics on combinations of categorical features and combinations of categorical and numerical features.

The method of transforming categorical features to numerical features includes the following stages:
1. Permutating the set of input objects in a random order.
2. Converting the label value from a floating point to an integer. (for regression: "Binarization is performed on the label value. The mode and number of buckets ($ k + 1 $ ) are set in the starting parameters. All values are located inside a single bucket)
3. Transforming categorical features to numerical features.<br>
<br>

 $$ \Large ctr_i=\frac{CountInClass + prior}{TotalCount + 1} $$


 * Borders: <br>
**`countInClass`** is how many times the label value exceeded $ \large i $ for objects with the current categorical feature value. It is only counts objects that have been made in the order of the objects after shuffling. <br>
**`totalCount`** is the total number of objects (up to the current one) that have a feature value matching the current one.<br>
**`prior`** is defined by the starting parameters.<br>


 * Buckets:<br>
**`countInClass`** is how many times the label value was equal to $ \large i $ for objects with the current categorical feature value. It is only counts objects that have been made in the order of the objects after shuffling. <br>
**`totalCount`** is the total number of objects (up to the current one) that have a feature value matching the current one.<br>
**`prior`** is defined by the starting parameters.<br>
    
    
  * BinarizedTargetMeanValue:<br>
**`countInClass`** is the ratio of the sum of the label value integers for this categorical feature to the maximum label value integer.<br>
**`totalCount`** is the total number of objects that have a feature value matching the current one.<br>
**`prior`** is defined by the starting parameters.<br>


 * Counter<br>
$$ \Large ctr_i=\frac{CurCount + prior}{TotalCount + 1} $$ 
**`curCount`** is the total number of objects in the training dataset with the current categorical feature value.<br>
**`maxCount`** the number of objects in the training dataset with the most frequent feature value.<br>
**`prior`** is defined by the starting parameters.<br>

# Comparing Hyperparameters

<img src='https://cdn-images-1.medium.com/max/2000/1*A0b_ahXOrrijazzJengwYw.png' aligh='center'>

### CatBoost parameters tunning

**`loss_function`** – The metric to use in training (the same as objective in xgb/lgb)<br>
**`eval_metric`** – The metric used for overfitting detection (if enabled) and the best model selection (if enabled).<br>
**`iterations`** – The maximum number of trees<br>
**`learning_rate`** – Learing rate :) (default 0.03)<br>
**`random_seed`** – is not set by default. always set to reproduce results.<br>
**`subsample`** – Sample rate for bagging (default 0.66)<br>
**`use_best_model`** – True if a validation set is input (the eval_set parameter is defined) and at least one of the values of objects in this set. False otherwise<br>
**`depth`** – the same as max depth earlier<br>
**`rsm`** – Random subspace method. The percentage of features that can be used at each split selection. (colsample_bylevel)<br>
**`class_weights`** – Classes weights<br>
**`od_type`** – The type of the overfitting detector to use. (better to use Iter)<br>
**`od_wait`** – The number of iterations to continue the training after the metric value.<br>






In [22]:
import catboost as ctb
parameters = {
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 10000,
    'learning_rate': 0.03,
    'random_seed': 42,
    'use_best_model': False,
    'od_wait': 30,
    'od_type': 'Iter'
}

In [23]:
ctb_data = ctb.Pool(X,y)
ctb.cv(ctb_data, parameters, stratified=True, seed=42, logging_level='Verbose')

/usr/local/lib/python3.6/site-packages/catboost/core.py:2064: UserWarning: Parameter "use_best_model" has no effect in cross-validation and is ignored
  warnings.warn('Parameter "use_best_model" has no effect in cross-validation and is ignored')


0:	learn: 0.7893936	test: 0.7886847	best: 0.7886847 (0)	total: 138ms	remaining: 23m 1s
1:	learn: 0.7990686	test: 0.7982627	best: 0.7982627 (1)	total: 227ms	remaining: 18m 56s
2:	learn: 0.7968766	test: 0.7960466	best: 0.7982627 (1)	total: 307ms	remaining: 17m 4s
3:	learn: 0.7978076	test: 0.7968858	best: 0.7982627 (1)	total: 403ms	remaining: 16m 47s
4:	learn: 0.7979959	test: 0.7969270	best: 0.7982627 (1)	total: 495ms	remaining: 16m 28s
5:	learn: 0.8163579	test: 0.8154051	best: 0.8154051 (5)	total: 647ms	remaining: 17m 57s
6:	learn: 0.8181118	test: 0.8166629	best: 0.8166629 (6)	total: 741ms	remaining: 17m 38s
7:	learn: 0.8180733	test: 0.8166377	best: 0.8166629 (6)	total: 831ms	remaining: 17m 18s
8:	learn: 0.8194125	test: 0.8178514	best: 0.8178514 (8)	total: 925ms	remaining: 17m 6s
9:	learn: 0.8199713	test: 0.8185492	best: 0.8185492 (9)	total: 1.02s	remaining: 16m 55s
10:	learn: 0.8200087	test: 0.8185769	best: 0.8185769 (10)	total: 1.1s	remaining: 16m 36s
11:	learn: 0.8211412	test: 0.81993

93:	learn: 0.8337841	test: 0.8309860	best: 0.8309860 (93)	total: 9.55s	remaining: 16m 46s
94:	learn: 0.8339176	test: 0.8310990	best: 0.8310990 (94)	total: 9.65s	remaining: 16m 46s
95:	learn: 0.8340069	test: 0.8311575	best: 0.8311575 (95)	total: 9.74s	remaining: 16m 45s
96:	learn: 0.8340647	test: 0.8311823	best: 0.8311823 (96)	total: 9.85s	remaining: 16m 45s
97:	learn: 0.8340765	test: 0.8311618	best: 0.8311823 (96)	total: 9.96s	remaining: 16m 46s
98:	learn: 0.8342398	test: 0.8312466	best: 0.8312466 (98)	total: 10.1s	remaining: 16m 48s
99:	learn: 0.8343323	test: 0.8312564	best: 0.8312564 (99)	total: 10.2s	remaining: 16m 46s
100:	learn: 0.8344975	test: 0.8313895	best: 0.8313895 (100)	total: 10.3s	remaining: 16m 44s
101:	learn: 0.8345805	test: 0.8314445	best: 0.8314445 (101)	total: 10.3s	remaining: 16m 43s
102:	learn: 0.8346935	test: 0.8315273	best: 0.8315273 (102)	total: 10.5s	remaining: 16m 48s
103:	learn: 0.8347919	test: 0.8315953	best: 0.8315953 (103)	total: 10.7s	remaining: 16m 56s
10

184:	learn: 0.8388859	test: 0.8339993	best: 0.8339993 (184)	total: 19s	remaining: 16m 46s
185:	learn: 0.8389368	test: 0.8340138	best: 0.8340138 (185)	total: 19.1s	remaining: 16m 47s
186:	learn: 0.8389693	test: 0.8340506	best: 0.8340506 (186)	total: 19.2s	remaining: 16m 48s
187:	learn: 0.8390053	test: 0.8340592	best: 0.8340592 (187)	total: 19.3s	remaining: 16m 47s
188:	learn: 0.8390445	test: 0.8340730	best: 0.8340730 (188)	total: 19.4s	remaining: 16m 46s
189:	learn: 0.8390830	test: 0.8340975	best: 0.8340975 (189)	total: 19.5s	remaining: 16m 46s
190:	learn: 0.8391305	test: 0.8341279	best: 0.8341279 (190)	total: 19.6s	remaining: 16m 47s
191:	learn: 0.8391852	test: 0.8341655	best: 0.8341655 (191)	total: 19.7s	remaining: 16m 46s
192:	learn: 0.8392133	test: 0.8341840	best: 0.8341840 (192)	total: 19.8s	remaining: 16m 45s
193:	learn: 0.8392592	test: 0.8341982	best: 0.8341982 (193)	total: 19.9s	remaining: 16m 44s
194:	learn: 0.8393008	test: 0.8342404	best: 0.8342404 (194)	total: 20s	remaining: 

274:	learn: 0.8414047	test: 0.8350983	best: 0.8350983 (274)	total: 28.1s	remaining: 16m 33s
275:	learn: 0.8414216	test: 0.8351093	best: 0.8351093 (275)	total: 28.2s	remaining: 16m 34s
276:	learn: 0.8414408	test: 0.8351264	best: 0.8351264 (276)	total: 28.3s	remaining: 16m 33s
277:	learn: 0.8414665	test: 0.8351333	best: 0.8351333 (277)	total: 28.4s	remaining: 16m 33s
278:	learn: 0.8414927	test: 0.8351404	best: 0.8351404 (278)	total: 28.5s	remaining: 16m 33s
279:	learn: 0.8415080	test: 0.8351405	best: 0.8351405 (279)	total: 28.6s	remaining: 16m 32s
280:	learn: 0.8415213	test: 0.8351393	best: 0.8351405 (279)	total: 28.7s	remaining: 16m 32s
281:	learn: 0.8415437	test: 0.8351461	best: 0.8351461 (281)	total: 28.8s	remaining: 16m 31s
282:	learn: 0.8415686	test: 0.8351576	best: 0.8351576 (282)	total: 28.9s	remaining: 16m 31s
283:	learn: 0.8415917	test: 0.8351668	best: 0.8351668 (283)	total: 29s	remaining: 16m 31s
284:	learn: 0.8416175	test: 0.8351608	best: 0.8351668 (283)	total: 29.1s	remaining

364:	learn: 0.8434519	test: 0.8356440	best: 0.8356440 (364)	total: 37.3s	remaining: 16m 24s
365:	learn: 0.8434837	test: 0.8356454	best: 0.8356454 (365)	total: 37.5s	remaining: 16m 26s
366:	learn: 0.8435019	test: 0.8356538	best: 0.8356538 (366)	total: 37.6s	remaining: 16m 26s
367:	learn: 0.8435330	test: 0.8356707	best: 0.8356707 (367)	total: 37.7s	remaining: 16m 26s
368:	learn: 0.8435542	test: 0.8356770	best: 0.8356770 (368)	total: 37.8s	remaining: 16m 26s
369:	learn: 0.8435748	test: 0.8356886	best: 0.8356886 (369)	total: 37.9s	remaining: 16m 25s
370:	learn: 0.8436048	test: 0.8357046	best: 0.8357046 (370)	total: 38s	remaining: 16m 27s
371:	learn: 0.8436286	test: 0.8357057	best: 0.8357057 (371)	total: 38.1s	remaining: 16m 27s
372:	learn: 0.8436472	test: 0.8357187	best: 0.8357187 (372)	total: 38.2s	remaining: 16m 26s
373:	learn: 0.8436604	test: 0.8357317	best: 0.8357317 (373)	total: 38.3s	remaining: 16m 26s
374:	learn: 0.8436861	test: 0.8357450	best: 0.8357450 (374)	total: 38.4s	remaining

455:	learn: 0.8456295	test: 0.8361058	best: 0.8361058 (455)	total: 46.2s	remaining: 16m 6s
456:	learn: 0.8456482	test: 0.8361093	best: 0.8361093 (456)	total: 46.3s	remaining: 16m 6s
457:	learn: 0.8456791	test: 0.8361144	best: 0.8361144 (457)	total: 46.4s	remaining: 16m 5s
458:	learn: 0.8456940	test: 0.8361298	best: 0.8361298 (458)	total: 46.5s	remaining: 16m 5s
459:	learn: 0.8457209	test: 0.8361263	best: 0.8361298 (458)	total: 46.5s	remaining: 16m 5s
460:	learn: 0.8457394	test: 0.8361288	best: 0.8361298 (458)	total: 46.6s	remaining: 16m 5s
461:	learn: 0.8457618	test: 0.8361278	best: 0.8361298 (458)	total: 46.7s	remaining: 16m 5s
462:	learn: 0.8457809	test: 0.8361349	best: 0.8361349 (462)	total: 46.8s	remaining: 16m 4s
463:	learn: 0.8458011	test: 0.8361483	best: 0.8361483 (463)	total: 46.9s	remaining: 16m 4s
464:	learn: 0.8458175	test: 0.8361421	best: 0.8361483 (463)	total: 47s	remaining: 16m 4s
465:	learn: 0.8458488	test: 0.8361415	best: 0.8361483 (463)	total: 47.1s	remaining: 16m 3s
4

546:	learn: 0.8474411	test: 0.8363907	best: 0.8364018 (540)	total: 54.9s	remaining: 15m 48s
547:	learn: 0.8474496	test: 0.8363832	best: 0.8364018 (540)	total: 55s	remaining: 15m 48s
548:	learn: 0.8474667	test: 0.8363778	best: 0.8364018 (540)	total: 55s	remaining: 15m 47s
549:	learn: 0.8474895	test: 0.8363874	best: 0.8364018 (540)	total: 55.1s	remaining: 15m 47s
550:	learn: 0.8475153	test: 0.8363868	best: 0.8364018 (540)	total: 55.2s	remaining: 15m 47s
551:	learn: 0.8475304	test: 0.8363908	best: 0.8364018 (540)	total: 55.3s	remaining: 15m 47s
552:	learn: 0.8475489	test: 0.8363962	best: 0.8364018 (540)	total: 55.5s	remaining: 15m 47s
553:	learn: 0.8475606	test: 0.8363953	best: 0.8364018 (540)	total: 55.5s	remaining: 15m 46s
554:	learn: 0.8475683	test: 0.8363881	best: 0.8364018 (540)	total: 55.6s	remaining: 15m 46s
555:	learn: 0.8475927	test: 0.8363948	best: 0.8364018 (540)	total: 55.7s	remaining: 15m 46s
556:	learn: 0.8476133	test: 0.8363938	best: 0.8364018 (540)	total: 55.8s	remaining: 

638:	learn: 0.8490624	test: 0.8364800	best: 0.8365027 (618)	total: 1m 3s	remaining: 15m 31s
639:	learn: 0.8490698	test: 0.8364777	best: 0.8365027 (618)	total: 1m 3s	remaining: 15m 31s
640:	learn: 0.8490856	test: 0.8364723	best: 0.8365027 (618)	total: 1m 3s	remaining: 15m 30s
641:	learn: 0.8491027	test: 0.8364669	best: 0.8365027 (618)	total: 1m 3s	remaining: 15m 30s
642:	learn: 0.8491237	test: 0.8364668	best: 0.8365027 (618)	total: 1m 3s	remaining: 15m 30s
643:	learn: 0.8491342	test: 0.8364704	best: 0.8365027 (618)	total: 1m 4s	remaining: 15m 30s
644:	learn: 0.8491579	test: 0.8364687	best: 0.8365027 (618)	total: 1m 4s	remaining: 15m 30s
645:	learn: 0.8491702	test: 0.8364715	best: 0.8365027 (618)	total: 1m 4s	remaining: 15m 29s
646:	learn: 0.8491872	test: 0.8364765	best: 0.8365027 (618)	total: 1m 4s	remaining: 15m 29s
647:	learn: 0.8491968	test: 0.8364739	best: 0.8365027 (618)	total: 1m 4s	remaining: 15m 29s
648:	learn: 0.8492121	test: 0.8364721	best: 0.8365027 (618)	total: 1m 4s	remaini

,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-AUC-mean,train-AUC-std,train-Logloss-mean,train-Logloss-std
0,0.788685,0.004977,0.673115,0.000069,0.789394,0.002519,0.673080,0.000173
1,0.798263,0.006230,0.654462,0.000126,0.799069,0.004227,0.654396,0.000425
2,0.796047,0.007604,0.637239,0.000162,0.796877,0.002754,0.637153,0.000574
3,0.796886,0.008361,0.621216,0.000202,0.797808,0.002851,0.621091,0.000709
4,0.796927,0.007508,0.606351,0.000243,0.797996,0.002958,0.606205,0.000820
5,0.815405,0.007919,0.591356,0.000270,0.816358,0.004371,0.591175,0.001062
6,0.816663,0.007954,0.578717,0.000569,0.818112,0.004285,0.578463,0.000793
7,0.816638,0.007952,0.566827,0.000596,0.818073,0.004261,0.566557,0.000917
8,0.817851,0.007684,0.555358,0.001001,0.819412,0.004045,0.555048,0.001082
9,0.818549,0.007673,0.544841,0.000886,0.819971,0.004152,0.544496,0.001187


In [24]:
parameters = {
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 650,
    'learning_rate': 0.03,
    'random_seed': 42,
    'use_best_model': False,
    'od_wait': 1000,
    'od_type': 'Iter'
}
for train, val in skf.split(X,y):
    temp_ctb_train = ctb.Pool(X.loc[train], y.loc[train])
    temp_ctb_val = ctb.Pool(X.loc[val], y.loc[val])
    temp_model = ctb.train(temp_ctb_train, parameters, logging_level='Verbose', eval_set=temp_ctb_val)

0:	learn: 0.7958939	test: 0.8006317	best: 0.8006317 (0)	total: 43.9ms	remaining: 28.5s
1:	learn: 0.8187609	test: 0.8216920	best: 0.8216920 (1)	total: 86.3ms	remaining: 28s
2:	learn: 0.8195932	test: 0.8195419	best: 0.8216920 (1)	total: 132ms	remaining: 28.6s
3:	learn: 0.8221195	test: 0.8222099	best: 0.8222099 (3)	total: 175ms	remaining: 28.3s
4:	learn: 0.8219652	test: 0.8221467	best: 0.8222099 (3)	total: 214ms	remaining: 27.6s
5:	learn: 0.8248168	test: 0.8247763	best: 0.8247763 (5)	total: 257ms	remaining: 27.6s
6:	learn: 0.8236099	test: 0.8241020	best: 0.8247763 (5)	total: 295ms	remaining: 27.1s
7:	learn: 0.8235291	test: 0.8237106	best: 0.8247763 (5)	total: 326ms	remaining: 26.2s
8:	learn: 0.8230093	test: 0.8233612	best: 0.8247763 (5)	total: 361ms	remaining: 25.7s
9:	learn: 0.8229671	test: 0.8233707	best: 0.8247763 (5)	total: 417ms	remaining: 26.7s
10:	learn: 0.8231561	test: 0.8234303	best: 0.8247763 (5)	total: 473ms	remaining: 27.5s
11:	learn: 0.8231202	test: 0.8234459	best: 0.8247763 

99:	learn: 0.8340775	test: 0.8314906	best: 0.8314906 (99)	total: 3.57s	remaining: 19.6s
100:	learn: 0.8340747	test: 0.8314958	best: 0.8314958 (100)	total: 3.6s	remaining: 19.5s
101:	learn: 0.8341932	test: 0.8316225	best: 0.8316225 (101)	total: 3.63s	remaining: 19.5s
102:	learn: 0.8342062	test: 0.8316143	best: 0.8316225 (101)	total: 3.66s	remaining: 19.4s
103:	learn: 0.8343588	test: 0.8317733	best: 0.8317733 (103)	total: 3.69s	remaining: 19.3s
104:	learn: 0.8344968	test: 0.8318110	best: 0.8318110 (104)	total: 3.73s	remaining: 19.3s
105:	learn: 0.8344821	test: 0.8317765	best: 0.8318110 (104)	total: 3.77s	remaining: 19.4s
106:	learn: 0.8346132	test: 0.8319381	best: 0.8319381 (106)	total: 3.82s	remaining: 19.4s
107:	learn: 0.8347012	test: 0.8320176	best: 0.8320176 (107)	total: 3.87s	remaining: 19.4s
108:	learn: 0.8347765	test: 0.8321048	best: 0.8321048 (108)	total: 3.91s	remaining: 19.4s
109:	learn: 0.8347929	test: 0.8320909	best: 0.8321048 (108)	total: 3.94s	remaining: 19.4s
110:	learn: 0

192:	learn: 0.8385753	test: 0.8345812	best: 0.8345812 (192)	total: 6.63s	remaining: 15.7s
193:	learn: 0.8386048	test: 0.8346083	best: 0.8346083 (193)	total: 6.68s	remaining: 15.7s
194:	learn: 0.8386852	test: 0.8346376	best: 0.8346376 (194)	total: 6.71s	remaining: 15.7s
195:	learn: 0.8386900	test: 0.8346509	best: 0.8346509 (195)	total: 6.74s	remaining: 15.6s
196:	learn: 0.8387331	test: 0.8346722	best: 0.8346722 (196)	total: 6.77s	remaining: 15.6s
197:	learn: 0.8387660	test: 0.8346814	best: 0.8346814 (197)	total: 6.81s	remaining: 15.5s
198:	learn: 0.8387823	test: 0.8346830	best: 0.8346830 (198)	total: 6.85s	remaining: 15.5s
199:	learn: 0.8387816	test: 0.8346746	best: 0.8346830 (198)	total: 6.89s	remaining: 15.5s
200:	learn: 0.8388376	test: 0.8347142	best: 0.8347142 (200)	total: 6.91s	remaining: 15.4s
201:	learn: 0.8389589	test: 0.8347858	best: 0.8347858 (201)	total: 6.94s	remaining: 15.4s
202:	learn: 0.8389974	test: 0.8348010	best: 0.8348010 (202)	total: 6.97s	remaining: 15.4s
203:	learn

285:	learn: 0.8412249	test: 0.8357035	best: 0.8357357 (283)	total: 10.2s	remaining: 13s
286:	learn: 0.8412290	test: 0.8356986	best: 0.8357357 (283)	total: 10.3s	remaining: 13s
287:	learn: 0.8412660	test: 0.8357497	best: 0.8357497 (287)	total: 10.3s	remaining: 13s
288:	learn: 0.8412993	test: 0.8357621	best: 0.8357621 (288)	total: 10.4s	remaining: 13s
289:	learn: 0.8413597	test: 0.8357901	best: 0.8357901 (289)	total: 10.4s	remaining: 13s
290:	learn: 0.8413902	test: 0.8357841	best: 0.8357901 (289)	total: 10.5s	remaining: 12.9s
291:	learn: 0.8414118	test: 0.8357789	best: 0.8357901 (289)	total: 10.5s	remaining: 12.9s
292:	learn: 0.8414379	test: 0.8357784	best: 0.8357901 (289)	total: 10.5s	remaining: 12.8s
293:	learn: 0.8414797	test: 0.8358022	best: 0.8358022 (293)	total: 10.6s	remaining: 12.8s
294:	learn: 0.8414898	test: 0.8358144	best: 0.8358144 (294)	total: 10.6s	remaining: 12.8s
295:	learn: 0.8415194	test: 0.8358249	best: 0.8358249 (295)	total: 10.6s	remaining: 12.7s
296:	learn: 0.841536

380:	learn: 0.8437114	test: 0.8364439	best: 0.8364439 (379)	total: 14s	remaining: 9.87s
381:	learn: 0.8437303	test: 0.8364544	best: 0.8364544 (381)	total: 14s	remaining: 9.85s
382:	learn: 0.8437483	test: 0.8364561	best: 0.8364561 (382)	total: 14.1s	remaining: 9.81s
383:	learn: 0.8437937	test: 0.8364592	best: 0.8364592 (383)	total: 14.1s	remaining: 9.78s
384:	learn: 0.8438045	test: 0.8364493	best: 0.8364592 (383)	total: 14.2s	remaining: 9.74s
385:	learn: 0.8438363	test: 0.8364286	best: 0.8364592 (383)	total: 14.2s	remaining: 9.71s
386:	learn: 0.8438363	test: 0.8364285	best: 0.8364592 (383)	total: 14.2s	remaining: 9.68s
387:	learn: 0.8438363	test: 0.8364283	best: 0.8364592 (383)	total: 14.3s	remaining: 9.63s
388:	learn: 0.8438514	test: 0.8364223	best: 0.8364592 (383)	total: 14.3s	remaining: 9.61s
389:	learn: 0.8438637	test: 0.8364272	best: 0.8364592 (383)	total: 14.4s	remaining: 9.58s
390:	learn: 0.8439030	test: 0.8364425	best: 0.8364592 (383)	total: 14.4s	remaining: 9.55s
391:	learn: 0.

...

222:	learn: 0.8380186	test: 0.8380029	best: 0.8380029 (222)	total: 7.27s	remaining: 13.9s
223:	learn: 0.8380149	test: 0.8380080	best: 0.8380080 (223)	total: 7.29s	remaining: 13.9s
224:	learn: 0.8380357	test: 0.8380143	best: 0.8380143 (224)	total: 7.32s	remaining: 13.8s
225:	learn: 0.8380512	test: 0.8380224	best: 0.8380224 (225)	total: 7.35s	remaining: 13.8s
226:	learn: 0.8380785	test: 0.8380313	best: 0.8380313 (226)	total: 7.38s	remaining: 13.8s
227:	learn: 0.8380787	test: 0.8380313	best: 0.8380313 (226)	total: 7.4s	remaining: 13.7s
228:	learn: 0.8381090	test: 0.8380188	best: 0.8380313 (226)	total: 7.43s	remaining: 13.7s
229:	learn: 0.8381148	test: 0.8380295	best: 0.8380313 (226)	total: 7.46s	remaining: 13.6s
230:	learn: 0.8381424	test: 0.8380213	best: 0.8380313 (226)	total: 7.49s	remaining: 13.6s
231:	learn: 0.8381635	test: 0.8380144	best: 0.8380313 (226)	total: 7.52s	remaining: 13.6s
232:	learn: 0.8381620	test: 0.8380111	best: 0.8380313 (226)	total: 7.55s	remaining: 13.5s
233:	learn:

314:	learn: 0.8405782	test: 0.8386523	best: 0.8386591 (313)	total: 10s	remaining: 10.7s
315:	learn: 0.8405861	test: 0.8386702	best: 0.8386702 (315)	total: 10.1s	remaining: 10.6s
316:	learn: 0.8405953	test: 0.8386741	best: 0.8386741 (316)	total: 10.1s	remaining: 10.6s
317:	learn: 0.8406141	test: 0.8386804	best: 0.8386804 (317)	total: 10.1s	remaining: 10.6s
318:	learn: 0.8406450	test: 0.8387057	best: 0.8387057 (318)	total: 10.1s	remaining: 10.5s
319:	learn: 0.8406566	test: 0.8387162	best: 0.8387162 (319)	total: 10.2s	remaining: 10.5s
320:	learn: 0.8406891	test: 0.8387294	best: 0.8387294 (320)	total: 10.2s	remaining: 10.4s
321:	learn: 0.8407172	test: 0.8387127	best: 0.8387294 (320)	total: 10.2s	remaining: 10.4s
322:	learn: 0.8407325	test: 0.8387106	best: 0.8387294 (320)	total: 10.3s	remaining: 10.4s
323:	learn: 0.8407477	test: 0.8387059	best: 0.8387294 (320)	total: 10.3s	remaining: 10.4s
324:	learn: 0.8407721	test: 0.8387230	best: 0.8387294 (320)	total: 10.3s	remaining: 10.3s
325:	learn: 

411:	learn: 0.8432381	test: 0.8393054	best: 0.8393087 (410)	total: 12.9s	remaining: 7.47s
412:	learn: 0.8432781	test: 0.8393167	best: 0.8393167 (412)	total: 13s	remaining: 7.43s
413:	learn: 0.8433084	test: 0.8393266	best: 0.8393266 (413)	total: 13s	remaining: 7.4s
414:	learn: 0.8433492	test: 0.8393350	best: 0.8393350 (414)	total: 13s	remaining: 7.37s
415:	learn: 0.8433671	test: 0.8393339	best: 0.8393350 (414)	total: 13s	remaining: 7.34s
416:	learn: 0.8433944	test: 0.8393275	best: 0.8393350 (414)	total: 13.1s	remaining: 7.3s
417:	learn: 0.8434316	test: 0.8393049	best: 0.8393350 (414)	total: 13.1s	remaining: 7.27s
418:	learn: 0.8434599	test: 0.8393005	best: 0.8393350 (414)	total: 13.1s	remaining: 7.24s
419:	learn: 0.8434882	test: 0.8392844	best: 0.8393350 (414)	total: 13.2s	remaining: 7.21s
420:	learn: 0.8435390	test: 0.8392998	best: 0.8393350 (414)	total: 13.2s	remaining: 7.18s
421:	learn: 0.8435928	test: 0.8392924	best: 0.8393350 (414)	total: 13.2s	remaining: 7.15s
422:	learn: 0.843636

503:	learn: 0.8457185	test: 0.8394108	best: 0.8394158 (492)	total: 16.1s	remaining: 4.67s
504:	learn: 0.8457493	test: 0.8394138	best: 0.8394158 (492)	total: 16.2s	remaining: 4.64s
505:	learn: 0.8457675	test: 0.8394091	best: 0.8394158 (492)	total: 16.2s	remaining: 4.61s
506:	learn: 0.8457966	test: 0.8394008	best: 0.8394158 (492)	total: 16.2s	remaining: 4.58s
507:	learn: 0.8458074	test: 0.8394156	best: 0.8394158 (492)	total: 16.3s	remaining: 4.55s
508:	learn: 0.8458235	test: 0.8394234	best: 0.8394234 (508)	total: 16.3s	remaining: 4.51s
509:	learn: 0.8458233	test: 0.8394222	best: 0.8394234 (508)	total: 16.3s	remaining: 4.48s
510:	learn: 0.8458462	test: 0.8394051	best: 0.8394234 (508)	total: 16.4s	remaining: 4.45s
511:	learn: 0.8458804	test: 0.8394016	best: 0.8394234 (508)	total: 16.4s	remaining: 4.42s
512:	learn: 0.8459078	test: 0.8394050	best: 0.8394234 (508)	total: 16.4s	remaining: 4.39s
513:	learn: 0.8459268	test: 0.8393992	best: 0.8394234 (508)	total: 16.5s	remaining: 4.36s
514:	learn

597:	learn: 0.8476989	test: 0.8396297	best: 0.8396336 (596)	total: 19.5s	remaining: 1.69s
598:	learn: 0.8477135	test: 0.8396346	best: 0.8396346 (598)	total: 19.5s	remaining: 1.66s
599:	learn: 0.8477158	test: 0.8396354	best: 0.8396354 (599)	total: 19.5s	remaining: 1.63s
600:	learn: 0.8477301	test: 0.8396265	best: 0.8396354 (599)	total: 19.6s	remaining: 1.59s
601:	learn: 0.8477544	test: 0.8396312	best: 0.8396354 (599)	total: 19.6s	remaining: 1.56s
602:	learn: 0.8477823	test: 0.8396292	best: 0.8396354 (599)	total: 19.6s	remaining: 1.53s
603:	learn: 0.8478148	test: 0.8396407	best: 0.8396407 (603)	total: 19.6s	remaining: 1.5s
604:	learn: 0.8478361	test: 0.8396385	best: 0.8396407 (603)	total: 19.7s	remaining: 1.46s
605:	learn: 0.8478737	test: 0.8396407	best: 0.8396407 (605)	total: 19.7s	remaining: 1.43s
606:	learn: 0.8478928	test: 0.8396408	best: 0.8396408 (606)	total: 19.7s	remaining: 1.4s
607:	learn: 0.8479144	test: 0.8396359	best: 0.8396408 (606)	total: 19.8s	remaining: 1.36s
608:	learn: 

In [25]:
(0.8367353+0.8334542+0.8396463441)/3.

0.8366119480333333

In [26]:
len(X.columns)
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [27]:
X.dtypes

age                                       int64
NumberOfTime30-59DaysPastDueNotWorse      int64
DebtRatio                               float64
NumberOfTimes90DaysLate                   int64
NumberOfTime60-89DaysPastDueNotWorse      int64
MonthlyIncome                           float64
NumberOfDependents                      float64
dtype: object

In [28]:
categorical_features_indices

array([0, 1, 3, 4])

In [29]:
parameters = {
    #default
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 10000,
    'learning_rate': 0.03,
    'random_seed': 42,
    'use_best_model': False,
    'od_wait': 50,
    'od_type': 'Iter',
    
    #regularization
    'rsm': 0.8,
    'l2_leaf_reg': 2
    
}

ctb_data = ctb.Pool(X,y)
ctb.cv(ctb_data, parameters, stratified=True, seed=42, logging_level='Verbose')

/usr/local/lib/python3.6/site-packages/catboost/core.py:2064: UserWarning: Parameter "use_best_model" has no effect in cross-validation and is ignored
  warnings.warn('Parameter "use_best_model" has no effect in cross-validation and is ignored')


0:	learn: 0.7854664	test: 0.7860432	best: 0.7860432 (0)	total: 80ms	remaining: 13m 19s
1:	learn: 0.8100637	test: 0.8107870	best: 0.8107870 (1)	total: 156ms	remaining: 13m 1s
2:	learn: 0.8172831	test: 0.8170366	best: 0.8170366 (2)	total: 239ms	remaining: 13m 15s
3:	learn: 0.8179012	test: 0.8179995	best: 0.8179995 (3)	total: 343ms	remaining: 14m 16s
4:	learn: 0.8178330	test: 0.8180210	best: 0.8180210 (4)	total: 442ms	remaining: 14m 43s
5:	learn: 0.8221419	test: 0.8220357	best: 0.8220357 (5)	total: 526ms	remaining: 14m 36s
6:	learn: 0.8225078	test: 0.8220666	best: 0.8220666 (6)	total: 620ms	remaining: 14m 45s
7:	learn: 0.8224538	test: 0.8220946	best: 0.8220946 (7)	total: 698ms	remaining: 14m 31s
8:	learn: 0.8237936	test: 0.8230657	best: 0.8230657 (8)	total: 779ms	remaining: 14m 25s
9:	learn: 0.8243084	test: 0.8232531	best: 0.8232531 (9)	total: 877ms	remaining: 14m 36s
10:	learn: 0.8251740	test: 0.8241685	best: 0.8241685 (10)	total: 960ms	remaining: 14m 32s
11:	learn: 0.8248657	test: 0.823

94:	learn: 0.8335989	test: 0.8308652	best: 0.8308652 (94)	total: 8.98s	remaining: 15m 36s
95:	learn: 0.8336968	test: 0.8309378	best: 0.8309378 (95)	total: 9.07s	remaining: 15m 35s
96:	learn: 0.8337443	test: 0.8309506	best: 0.8309506 (96)	total: 9.15s	remaining: 15m 34s
97:	learn: 0.8338190	test: 0.8310172	best: 0.8310172 (97)	total: 9.25s	remaining: 15m 34s
98:	learn: 0.8339005	test: 0.8310385	best: 0.8310385 (98)	total: 9.34s	remaining: 15m 33s
99:	learn: 0.8339777	test: 0.8310973	best: 0.8310973 (99)	total: 9.42s	remaining: 15m 32s
100:	learn: 0.8339951	test: 0.8311073	best: 0.8311073 (100)	total: 9.51s	remaining: 15m 32s
101:	learn: 0.8340903	test: 0.8311768	best: 0.8311768 (101)	total: 9.6s	remaining: 15m 31s
102:	learn: 0.8341560	test: 0.8312253	best: 0.8312253 (102)	total: 9.69s	remaining: 15m 31s
103:	learn: 0.8342525	test: 0.8313271	best: 0.8313271 (103)	total: 9.78s	remaining: 15m 30s
104:	learn: 0.8342777	test: 0.8313457	best: 0.8313457 (104)	total: 9.86s	remaining: 15m 29s
1

185:	learn: 0.8384104	test: 0.8339066	best: 0.8339067 (184)	total: 18.2s	remaining: 16m
186:	learn: 0.8384514	test: 0.8339359	best: 0.8339359 (186)	total: 18.3s	remaining: 16m 2s
187:	learn: 0.8384857	test: 0.8339552	best: 0.8339552 (187)	total: 18.5s	remaining: 16m 5s
188:	learn: 0.8385036	test: 0.8339641	best: 0.8339641 (188)	total: 18.6s	remaining: 16m 6s
189:	learn: 0.8385259	test: 0.8339755	best: 0.8339755 (189)	total: 18.8s	remaining: 16m 12s
190:	learn: 0.8385684	test: 0.8340037	best: 0.8340037 (190)	total: 19s	remaining: 16m 14s
191:	learn: 0.8386041	test: 0.8340363	best: 0.8340363 (191)	total: 19.1s	remaining: 16m 15s
192:	learn: 0.8386463	test: 0.8340544	best: 0.8340544 (192)	total: 19.2s	remaining: 16m 16s
193:	learn: 0.8387022	test: 0.8340966	best: 0.8340966 (193)	total: 19.4s	remaining: 16m 18s
194:	learn: 0.8387535	test: 0.8341125	best: 0.8341125 (194)	total: 19.5s	remaining: 16m 19s
195:	learn: 0.8387916	test: 0.8341421	best: 0.8341421 (195)	total: 19.6s	remaining: 16m 1

275:	learn: 0.8411696	test: 0.8352105	best: 0.8352105 (275)	total: 27.5s	remaining: 16m 10s
276:	learn: 0.8411979	test: 0.8352186	best: 0.8352186 (276)	total: 27.6s	remaining: 16m 9s
277:	learn: 0.8412141	test: 0.8352152	best: 0.8352186 (276)	total: 27.7s	remaining: 16m 9s
278:	learn: 0.8412486	test: 0.8352355	best: 0.8352355 (278)	total: 27.8s	remaining: 16m 9s
279:	learn: 0.8412588	test: 0.8352376	best: 0.8352376 (279)	total: 27.9s	remaining: 16m 8s
280:	learn: 0.8412725	test: 0.8352441	best: 0.8352441 (280)	total: 28s	remaining: 16m 8s
281:	learn: 0.8413003	test: 0.8352548	best: 0.8352548 (281)	total: 28.1s	remaining: 16m 7s
282:	learn: 0.8413335	test: 0.8352715	best: 0.8352715 (282)	total: 28.2s	remaining: 16m 7s
283:	learn: 0.8413675	test: 0.8352810	best: 0.8352810 (283)	total: 28.3s	remaining: 16m 6s
284:	learn: 0.8414047	test: 0.8352983	best: 0.8352983 (284)	total: 28.4s	remaining: 16m 6s
285:	learn: 0.8414209	test: 0.8352927	best: 0.8352983 (284)	total: 28.4s	remaining: 16m 5s


...

457:	learn: 0.8454035	test: 0.8362470	best: 0.8362491 (456)	total: 45.4s	remaining: 15m 46s
458:	learn: 0.8454169	test: 0.8362473	best: 0.8362491 (456)	total: 45.5s	remaining: 15m 46s
459:	learn: 0.8454444	test: 0.8362516	best: 0.8362516 (459)	total: 45.6s	remaining: 15m 45s
460:	learn: 0.8454656	test: 0.8362532	best: 0.8362532 (460)	total: 45.7s	remaining: 15m 46s
461:	learn: 0.8454818	test: 0.8362592	best: 0.8362592 (461)	total: 45.8s	remaining: 15m 46s
462:	learn: 0.8455089	test: 0.8362645	best: 0.8362645 (462)	total: 46s	remaining: 15m 46s
463:	learn: 0.8455385	test: 0.8362678	best: 0.8362678 (463)	total: 46.1s	remaining: 15m 46s
464:	learn: 0.8455543	test: 0.8362732	best: 0.8362732 (464)	total: 46.2s	remaining: 15m 47s
465:	learn: 0.8455899	test: 0.8362916	best: 0.8362916 (465)	total: 46.3s	remaining: 15m 47s
466:	learn: 0.8456124	test: 0.8362971	best: 0.8362971 (466)	total: 46.4s	remaining: 15m 47s
467:	learn: 0.8456276	test: 0.8363084	best: 0.8363084 (467)	total: 46.5s	remaining

547:	learn: 0.8471497	test: 0.8365584	best: 0.8365584 (547)	total: 55.2s	remaining: 15m 52s
548:	learn: 0.8471594	test: 0.8365645	best: 0.8365645 (548)	total: 55.3s	remaining: 15m 52s
549:	learn: 0.8471737	test: 0.8365644	best: 0.8365645 (548)	total: 55.4s	remaining: 15m 52s
550:	learn: 0.8471974	test: 0.8365656	best: 0.8365656 (550)	total: 55.5s	remaining: 15m 51s
551:	learn: 0.8472166	test: 0.8365646	best: 0.8365656 (550)	total: 55.6s	remaining: 15m 51s
552:	learn: 0.8472316	test: 0.8365708	best: 0.8365708 (552)	total: 55.7s	remaining: 15m 51s
553:	learn: 0.8472525	test: 0.8365785	best: 0.8365785 (553)	total: 55.8s	remaining: 15m 51s
554:	learn: 0.8472592	test: 0.8365771	best: 0.8365785 (553)	total: 55.9s	remaining: 15m 51s
555:	learn: 0.8472730	test: 0.8365774	best: 0.8365785 (553)	total: 56s	remaining: 15m 51s
556:	learn: 0.8472909	test: 0.8365795	best: 0.8365795 (556)	total: 56.1s	remaining: 15m 51s
557:	learn: 0.8473030	test: 0.8365806	best: 0.8365806 (557)	total: 56.2s	remaining

637:	learn: 0.8487092	test: 0.8366616	best: 0.8366648 (633)	total: 1m 4s	remaining: 15m 42s
638:	learn: 0.8487286	test: 0.8366633	best: 0.8366648 (633)	total: 1m 4s	remaining: 15m 41s
639:	learn: 0.8487453	test: 0.8366683	best: 0.8366683 (639)	total: 1m 4s	remaining: 15m 41s
640:	learn: 0.8487664	test: 0.8366661	best: 0.8366683 (639)	total: 1m 4s	remaining: 15m 41s
641:	learn: 0.8487884	test: 0.8366562	best: 0.8366683 (639)	total: 1m 4s	remaining: 15m 41s
642:	learn: 0.8488016	test: 0.8366601	best: 0.8366683 (639)	total: 1m 4s	remaining: 15m 41s
643:	learn: 0.8488156	test: 0.8366599	best: 0.8366683 (639)	total: 1m 4s	remaining: 15m 41s
644:	learn: 0.8488380	test: 0.8366630	best: 0.8366683 (639)	total: 1m 4s	remaining: 15m 40s
645:	learn: 0.8488505	test: 0.8366578	best: 0.8366683 (639)	total: 1m 5s	remaining: 15m 41s
646:	learn: 0.8488700	test: 0.8366714	best: 0.8366714 (646)	total: 1m 5s	remaining: 15m 41s
647:	learn: 0.8488872	test: 0.8366755	best: 0.8366755 (647)	total: 1m 5s	remaini

727:	learn: 0.8500997	test: 0.8367350	best: 0.8367365 (712)	total: 1m 13s	remaining: 15m 35s
728:	learn: 0.8501154	test: 0.8367320	best: 0.8367365 (712)	total: 1m 13s	remaining: 15m 34s
729:	learn: 0.8501267	test: 0.8367341	best: 0.8367365 (712)	total: 1m 13s	remaining: 15m 35s
730:	learn: 0.8501469	test: 0.8367347	best: 0.8367365 (712)	total: 1m 13s	remaining: 15m 36s
731:	learn: 0.8501671	test: 0.8367345	best: 0.8367365 (712)	total: 1m 13s	remaining: 15m 35s
732:	learn: 0.8501874	test: 0.8367310	best: 0.8367365 (712)	total: 1m 14s	remaining: 15m 35s
733:	learn: 0.8502012	test: 0.8367375	best: 0.8367375 (733)	total: 1m 14s	remaining: 15m 35s
734:	learn: 0.8502172	test: 0.8367380	best: 0.8367380 (734)	total: 1m 14s	remaining: 15m 35s
735:	learn: 0.8502269	test: 0.8367351	best: 0.8367380 (734)	total: 1m 14s	remaining: 15m 35s
736:	learn: 0.8502569	test: 0.8367330	best: 0.8367380 (734)	total: 1m 14s	remaining: 15m 34s
737:	learn: 0.8502669	test: 0.8367361	best: 0.8367380 (734)	total: 1m 

,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-AUC-mean,train-AUC-std,train-Logloss-mean,train-Logloss-std
0,0.786043,0.004867,0.673103,0.000048,0.785466,0.003965,0.673073,0.000212
1,0.810787,0.008759,0.653746,0.000388,0.810064,0.004644,0.653690,0.000396
2,0.817037,0.008068,0.636235,0.000415,0.817283,0.002648,0.636146,0.000548
3,0.817999,0.008902,0.620419,0.000782,0.817901,0.003101,0.620286,0.000457
4,0.818021,0.008345,0.605451,0.000791,0.817833,0.002945,0.605296,0.000554
5,0.822036,0.008154,0.591324,0.000734,0.822142,0.004433,0.591127,0.000817
6,0.822067,0.006614,0.577801,0.000732,0.822508,0.004422,0.577569,0.000934
7,0.822095,0.006608,0.566546,0.000767,0.822454,0.004468,0.566294,0.001024
8,0.823066,0.006835,0.554498,0.000447,0.823794,0.004793,0.554166,0.001351
9,0.823253,0.006393,0.543760,0.000235,0.824308,0.004695,0.543368,0.001855


In [30]:
parameters = {
    #default
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 759,
    'learning_rate': 0.03,
    'random_seed': 42,
    'use_best_model': False,
    
    #regularization
    'rsm': 0.8,
    'l2_leaf_reg': 2
    
}

for train, val in skf.split(X,y):
    temp_ctb_train = ctb.Pool(X.loc[train], y.loc[train])
    temp_ctb_val = ctb.Pool(X.loc[val], y.loc[val])
    temp_model = ctb.train(temp_ctb_train, parameters, logging_level='Verbose', eval_set=temp_ctb_val)

0:	learn: 0.7755048	test: 0.7785565	best: 0.7785565 (0)	total: 45.6ms	remaining: 34.6s
1:	learn: 0.7791169	test: 0.7843768	best: 0.7843768 (1)	total: 85ms	remaining: 32.2s
2:	learn: 0.8032358	test: 0.8081077	best: 0.8081077 (2)	total: 113ms	remaining: 28.5s
3:	learn: 0.8046278	test: 0.8103115	best: 0.8103115 (3)	total: 151ms	remaining: 28.5s
4:	learn: 0.8160360	test: 0.8180919	best: 0.8180919 (4)	total: 181ms	remaining: 27.2s
5:	learn: 0.8179086	test: 0.8190544	best: 0.8190544 (5)	total: 213ms	remaining: 26.7s
6:	learn: 0.8178536	test: 0.8188539	best: 0.8190544 (5)	total: 244ms	remaining: 26.2s
7:	learn: 0.8177915	test: 0.8193381	best: 0.8193381 (7)	total: 281ms	remaining: 26.4s
8:	learn: 0.8192048	test: 0.8201869	best: 0.8201869 (8)	total: 314ms	remaining: 26.2s
9:	learn: 0.8193581	test: 0.8207333	best: 0.8207333 (9)	total: 343ms	remaining: 25.7s
10:	learn: 0.8198059	test: 0.8208013	best: 0.8208013 (10)	total: 372ms	remaining: 25.3s
11:	learn: 0.8197719	test: 0.8205483	best: 0.8208013

95:	learn: 0.8332468	test: 0.8313719	best: 0.8313719 (95)	total: 3.72s	remaining: 25.7s
96:	learn: 0.8332941	test: 0.8314484	best: 0.8314484 (96)	total: 3.76s	remaining: 25.6s
97:	learn: 0.8332893	test: 0.8314323	best: 0.8314484 (96)	total: 3.79s	remaining: 25.5s
98:	learn: 0.8334415	test: 0.8316364	best: 0.8316364 (98)	total: 3.81s	remaining: 25.4s
99:	learn: 0.8335650	test: 0.8317406	best: 0.8317406 (99)	total: 3.84s	remaining: 25.3s
100:	learn: 0.8335912	test: 0.8317416	best: 0.8317416 (100)	total: 3.87s	remaining: 25.2s
101:	learn: 0.8335959	test: 0.8317400	best: 0.8317416 (100)	total: 3.9s	remaining: 25.1s
102:	learn: 0.8336896	test: 0.8318192	best: 0.8318192 (102)	total: 3.93s	remaining: 25s
103:	learn: 0.8338877	test: 0.8320229	best: 0.8320229 (103)	total: 3.98s	remaining: 25s
104:	learn: 0.8340069	test: 0.8321219	best: 0.8321219 (104)	total: 4.01s	remaining: 25s
105:	learn: 0.8341061	test: 0.8322164	best: 0.8322164 (105)	total: 4.04s	remaining: 24.9s
106:	learn: 0.8342348	test:

187:	learn: 0.8381246	test: 0.8345909	best: 0.8345909 (187)	total: 6.79s	remaining: 20.6s
188:	learn: 0.8381265	test: 0.8345801	best: 0.8345909 (187)	total: 6.82s	remaining: 20.6s
189:	learn: 0.8382260	test: 0.8345870	best: 0.8345909 (187)	total: 6.85s	remaining: 20.5s
190:	learn: 0.8382471	test: 0.8346146	best: 0.8346146 (190)	total: 6.88s	remaining: 20.5s
191:	learn: 0.8382754	test: 0.8346580	best: 0.8346580 (191)	total: 6.92s	remaining: 20.4s
192:	learn: 0.8382915	test: 0.8346862	best: 0.8346862 (192)	total: 6.95s	remaining: 20.4s
193:	learn: 0.8383521	test: 0.8347300	best: 0.8347300 (193)	total: 6.98s	remaining: 20.3s
194:	learn: 0.8383793	test: 0.8347402	best: 0.8347402 (194)	total: 7.01s	remaining: 20.3s
195:	learn: 0.8384095	test: 0.8347721	best: 0.8347721 (195)	total: 7.05s	remaining: 20.3s
196:	learn: 0.8384564	test: 0.8348130	best: 0.8348130 (196)	total: 7.08s	remaining: 20.2s
197:	learn: 0.8384631	test: 0.8348080	best: 0.8348130 (196)	total: 7.12s	remaining: 20.2s
198:	learn

283:	learn: 0.8409519	test: 0.8356958	best: 0.8356958 (283)	total: 10.1s	remaining: 16.9s
284:	learn: 0.8409704	test: 0.8357242	best: 0.8357242 (284)	total: 10.1s	remaining: 16.8s
285:	learn: 0.8409987	test: 0.8357432	best: 0.8357432 (285)	total: 10.1s	remaining: 16.8s
286:	learn: 0.8410098	test: 0.8357294	best: 0.8357432 (285)	total: 10.2s	remaining: 16.7s
287:	learn: 0.8410420	test: 0.8357364	best: 0.8357432 (285)	total: 10.2s	remaining: 16.7s
288:	learn: 0.8410736	test: 0.8357532	best: 0.8357532 (288)	total: 10.2s	remaining: 16.6s
289:	learn: 0.8411083	test: 0.8357728	best: 0.8357728 (289)	total: 10.3s	remaining: 16.6s
290:	learn: 0.8411218	test: 0.8357763	best: 0.8357763 (290)	total: 10.3s	remaining: 16.6s
291:	learn: 0.8411335	test: 0.8357787	best: 0.8357787 (291)	total: 10.4s	remaining: 16.6s
292:	learn: 0.8411637	test: 0.8358079	best: 0.8358079 (292)	total: 10.4s	remaining: 16.6s
293:	learn: 0.8411956	test: 0.8358146	best: 0.8358146 (293)	total: 10.5s	remaining: 16.5s
294:	learn

376:	learn: 0.8432447	test: 0.8363962	best: 0.8364080 (375)	total: 14.3s	remaining: 14.5s
377:	learn: 0.8432595	test: 0.8363644	best: 0.8364080 (375)	total: 14.4s	remaining: 14.5s
378:	learn: 0.8432705	test: 0.8363780	best: 0.8364080 (375)	total: 14.4s	remaining: 14.5s
379:	learn: 0.8433059	test: 0.8363568	best: 0.8364080 (375)	total: 14.5s	remaining: 14.4s
380:	learn: 0.8433427	test: 0.8363732	best: 0.8364080 (375)	total: 14.5s	remaining: 14.4s
381:	learn: 0.8433658	test: 0.8363858	best: 0.8364080 (375)	total: 14.6s	remaining: 14.4s
382:	learn: 0.8434067	test: 0.8364100	best: 0.8364100 (382)	total: 14.7s	remaining: 14.4s
383:	learn: 0.8434251	test: 0.8364162	best: 0.8364162 (383)	total: 14.7s	remaining: 14.3s
384:	learn: 0.8434379	test: 0.8364303	best: 0.8364303 (384)	total: 14.7s	remaining: 14.3s
385:	learn: 0.8434493	test: 0.8364279	best: 0.8364303 (384)	total: 14.8s	remaining: 14.3s
386:	learn: 0.8434674	test: 0.8364209	best: 0.8364303 (384)	total: 14.8s	remaining: 14.2s
387:	learn

...

366:	learn: 0.8417871	test: 0.8391512	best: 0.8391512 (366)	total: 13.8s	remaining: 14.8s
367:	learn: 0.8418015	test: 0.8391478	best: 0.8391512 (366)	total: 13.8s	remaining: 14.7s
368:	learn: 0.8418349	test: 0.8391508	best: 0.8391512 (366)	total: 13.9s	remaining: 14.7s
369:	learn: 0.8418798	test: 0.8391366	best: 0.8391512 (366)	total: 13.9s	remaining: 14.6s
370:	learn: 0.8419180	test: 0.8391587	best: 0.8391587 (370)	total: 13.9s	remaining: 14.6s
371:	learn: 0.8419460	test: 0.8391704	best: 0.8391704 (371)	total: 14s	remaining: 14.5s
372:	learn: 0.8419708	test: 0.8391839	best: 0.8391839 (372)	total: 14s	remaining: 14.5s
373:	learn: 0.8420121	test: 0.8391872	best: 0.8391872 (373)	total: 14s	remaining: 14.5s
374:	learn: 0.8420341	test: 0.8391962	best: 0.8391962 (374)	total: 14.1s	remaining: 14.4s
375:	learn: 0.8420600	test: 0.8392091	best: 0.8392091 (375)	total: 14.2s	remaining: 14.4s
376:	learn: 0.8420807	test: 0.8392097	best: 0.8392097 (376)	total: 14.2s	remaining: 14.4s
377:	learn: 0.84

460:	learn: 0.8443748	test: 0.8396814	best: 0.8397041 (452)	total: 18s	remaining: 11.6s
461:	learn: 0.8443806	test: 0.8396803	best: 0.8397041 (452)	total: 18s	remaining: 11.6s
462:	learn: 0.8444292	test: 0.8396935	best: 0.8397041 (452)	total: 18s	remaining: 11.5s
463:	learn: 0.8444291	test: 0.8396912	best: 0.8397041 (452)	total: 18.1s	remaining: 11.5s
464:	learn: 0.8444317	test: 0.8396887	best: 0.8397041 (452)	total: 18.1s	remaining: 11.4s
465:	learn: 0.8444584	test: 0.8397078	best: 0.8397078 (465)	total: 18.1s	remaining: 11.4s
466:	learn: 0.8444587	test: 0.8397049	best: 0.8397078 (465)	total: 18.2s	remaining: 11.4s
467:	learn: 0.8444767	test: 0.8397066	best: 0.8397078 (465)	total: 18.2s	remaining: 11.3s
468:	learn: 0.8445006	test: 0.8397312	best: 0.8397312 (468)	total: 18.3s	remaining: 11.3s
469:	learn: 0.8445233	test: 0.8397208	best: 0.8397312 (468)	total: 18.3s	remaining: 11.2s
470:	learn: 0.8445457	test: 0.8397126	best: 0.8397312 (468)	total: 18.3s	remaining: 11.2s
471:	learn: 0.84

556:	learn: 0.8465357	test: 0.8397974	best: 0.8398436 (537)	total: 22.1s	remaining: 8s
557:	learn: 0.8465544	test: 0.8397949	best: 0.8398436 (537)	total: 22.1s	remaining: 7.96s
558:	learn: 0.8465665	test: 0.8397886	best: 0.8398436 (537)	total: 22.1s	remaining: 7.92s
559:	learn: 0.8465953	test: 0.8398008	best: 0.8398436 (537)	total: 22.2s	remaining: 7.88s
560:	learn: 0.8466306	test: 0.8398015	best: 0.8398436 (537)	total: 22.2s	remaining: 7.84s
561:	learn: 0.8466434	test: 0.8397967	best: 0.8398436 (537)	total: 22.3s	remaining: 7.81s
562:	learn: 0.8466617	test: 0.8397898	best: 0.8398436 (537)	total: 22.3s	remaining: 7.77s
563:	learn: 0.8466896	test: 0.8397839	best: 0.8398436 (537)	total: 22.4s	remaining: 7.73s
564:	learn: 0.8467090	test: 0.8397804	best: 0.8398436 (537)	total: 22.4s	remaining: 7.68s
565:	learn: 0.8467142	test: 0.8397735	best: 0.8398436 (537)	total: 22.4s	remaining: 7.64s
566:	learn: 0.8467411	test: 0.8397807	best: 0.8398436 (537)	total: 22.4s	remaining: 7.6s
567:	learn: 0.

652:	learn: 0.8482971	test: 0.8396609	best: 0.8398436 (537)	total: 25.6s	remaining: 4.15s
653:	learn: 0.8483154	test: 0.8396648	best: 0.8398436 (537)	total: 25.6s	remaining: 4.11s
654:	learn: 0.8483273	test: 0.8396533	best: 0.8398436 (537)	total: 25.7s	remaining: 4.07s
655:	learn: 0.8483474	test: 0.8396501	best: 0.8398436 (537)	total: 25.7s	remaining: 4.03s
656:	learn: 0.8483546	test: 0.8396515	best: 0.8398436 (537)	total: 25.7s	remaining: 3.99s
657:	learn: 0.8483814	test: 0.8396446	best: 0.8398436 (537)	total: 25.7s	remaining: 3.95s
658:	learn: 0.8483865	test: 0.8396518	best: 0.8398436 (537)	total: 25.8s	remaining: 3.91s
659:	learn: 0.8484031	test: 0.8396532	best: 0.8398436 (537)	total: 25.8s	remaining: 3.87s
660:	learn: 0.8484306	test: 0.8396563	best: 0.8398436 (537)	total: 25.8s	remaining: 3.83s
661:	learn: 0.8484447	test: 0.8396622	best: 0.8398436 (537)	total: 25.9s	remaining: 3.79s
662:	learn: 0.8484789	test: 0.8396556	best: 0.8398436 (537)	total: 25.9s	remaining: 3.75s
663:	learn

748:	learn: 0.8499738	test: 0.8395317	best: 0.8398436 (537)	total: 28.6s	remaining: 382ms
749:	learn: 0.8499990	test: 0.8395263	best: 0.8398436 (537)	total: 28.6s	remaining: 344ms
750:	learn: 0.8500047	test: 0.8395295	best: 0.8398436 (537)	total: 28.7s	remaining: 305ms
751:	learn: 0.8500095	test: 0.8395307	best: 0.8398436 (537)	total: 28.7s	remaining: 267ms
752:	learn: 0.8500209	test: 0.8395310	best: 0.8398436 (537)	total: 28.7s	remaining: 229ms
753:	learn: 0.8500332	test: 0.8395234	best: 0.8398436 (537)	total: 28.8s	remaining: 191ms
754:	learn: 0.8500408	test: 0.8395188	best: 0.8398436 (537)	total: 28.8s	remaining: 153ms
755:	learn: 0.8500528	test: 0.8395227	best: 0.8398436 (537)	total: 28.8s	remaining: 114ms
756:	learn: 0.8500658	test: 0.8395282	best: 0.8398436 (537)	total: 28.9s	remaining: 76.2ms
757:	learn: 0.8500759	test: 0.8395323	best: 0.8398436 (537)	total: 28.9s	remaining: 38.1ms
758:	learn: 0.8501085	test: 0.8395389	best: 0.8398436 (537)	total: 28.9s	remaining: 0us

bestTest 

In [31]:
(0.8369334 + 0.8337581 + 0.8395389)/3.

0.8367434666666668

In [32]:
# with cateforical features
for train, val in skf.split(X,y):
    ctb_model = ctb.CatBoostClassifier(iterations=1000, learning_rate=0.03, depth=6, eval_metric='AUC', 
                                       random_seed=42, rsm=0.8, l2_leaf_reg=2, od_wait=50, od_type='Iter')
    ctb_model.fit(X.loc[train], y.loc[train], cat_features=[6], use_best_model=True, eval_set=(X.loc[val], y.loc[val]), verbose=True)

0:	learn: 0.7753115	test: 0.7792025	best: 0.7792025 (0)	total: 31.6ms	remaining: 31.6s
1:	learn: 0.8075480	test: 0.8080354	best: 0.8080354 (1)	total: 62.8ms	remaining: 31.3s
2:	learn: 0.8088956	test: 0.8105515	best: 0.8105515 (2)	total: 92ms	remaining: 30.6s
3:	learn: 0.8139238	test: 0.8162055	best: 0.8162055 (3)	total: 121ms	remaining: 30.2s
4:	learn: 0.8145083	test: 0.8160978	best: 0.8162055 (3)	total: 154ms	remaining: 30.7s
5:	learn: 0.8178418	test: 0.8190590	best: 0.8190590 (5)	total: 184ms	remaining: 30.4s
6:	learn: 0.8180805	test: 0.8191328	best: 0.8191328 (6)	total: 214ms	remaining: 30.4s
7:	learn: 0.8182871	test: 0.8193753	best: 0.8193753 (7)	total: 276ms	remaining: 34.2s
8:	learn: 0.8183718	test: 0.8190858	best: 0.8193753 (7)	total: 306ms	remaining: 33.7s
9:	learn: 0.8184250	test: 0.8197682	best: 0.8197682 (9)	total: 337ms	remaining: 33.4s
10:	learn: 0.8187170	test: 0.8198956	best: 0.8198956 (10)	total: 367ms	remaining: 33s
11:	learn: 0.8187371	test: 0.8203402	best: 0.8203402 

94:	learn: 0.8326181	test: 0.8307161	best: 0.8307161 (94)	total: 3.57s	remaining: 34s
95:	learn: 0.8327282	test: 0.8308958	best: 0.8308958 (95)	total: 3.62s	remaining: 34.1s
96:	learn: 0.8327940	test: 0.8308786	best: 0.8308958 (95)	total: 3.66s	remaining: 34s
97:	learn: 0.8329715	test: 0.8310758	best: 0.8310758 (97)	total: 3.69s	remaining: 34s
98:	learn: 0.8329747	test: 0.8311102	best: 0.8311102 (98)	total: 3.72s	remaining: 33.9s
99:	learn: 0.8330329	test: 0.8310969	best: 0.8311102 (98)	total: 3.76s	remaining: 33.8s
100:	learn: 0.8331097	test: 0.8311213	best: 0.8311213 (100)	total: 3.8s	remaining: 33.9s
101:	learn: 0.8331720	test: 0.8311659	best: 0.8311659 (101)	total: 3.84s	remaining: 33.8s
102:	learn: 0.8332020	test: 0.8311117	best: 0.8311659 (101)	total: 3.87s	remaining: 33.7s
103:	learn: 0.8333980	test: 0.8312936	best: 0.8312936 (103)	total: 3.91s	remaining: 33.7s
104:	learn: 0.8334231	test: 0.8312806	best: 0.8312936 (103)	total: 3.94s	remaining: 33.6s
105:	learn: 0.8334492	test: 0

188:	learn: 0.8372121	test: 0.8337557	best: 0.8337692 (184)	total: 6.96s	remaining: 29.9s
189:	learn: 0.8372884	test: 0.8337739	best: 0.8337739 (189)	total: 7s	remaining: 29.8s
190:	learn: 0.8373484	test: 0.8338233	best: 0.8338233 (190)	total: 7.03s	remaining: 29.8s
191:	learn: 0.8373634	test: 0.8338547	best: 0.8338547 (191)	total: 7.07s	remaining: 29.7s
192:	learn: 0.8374134	test: 0.8338929	best: 0.8338929 (192)	total: 7.1s	remaining: 29.7s
193:	learn: 0.8374616	test: 0.8339079	best: 0.8339079 (193)	total: 7.13s	remaining: 29.6s
194:	learn: 0.8374917	test: 0.8338979	best: 0.8339079 (193)	total: 7.17s	remaining: 29.6s
195:	learn: 0.8375068	test: 0.8339064	best: 0.8339079 (193)	total: 7.2s	remaining: 29.5s
196:	learn: 0.8375532	test: 0.8339022	best: 0.8339079 (193)	total: 7.23s	remaining: 29.5s
197:	learn: 0.8375768	test: 0.8339363	best: 0.8339363 (197)	total: 7.27s	remaining: 29.4s
198:	learn: 0.8376279	test: 0.8339516	best: 0.8339516 (198)	total: 7.3s	remaining: 29.4s
199:	learn: 0.83

285:	learn: 0.8401456	test: 0.8352951	best: 0.8352951 (285)	total: 10.3s	remaining: 25.7s
286:	learn: 0.8401900	test: 0.8353059	best: 0.8353059 (286)	total: 10.3s	remaining: 25.6s
287:	learn: 0.8402153	test: 0.8353301	best: 0.8353301 (287)	total: 10.3s	remaining: 25.6s
288:	learn: 0.8402367	test: 0.8353192	best: 0.8353301 (287)	total: 10.4s	remaining: 25.5s
289:	learn: 0.8402537	test: 0.8353451	best: 0.8353451 (289)	total: 10.4s	remaining: 25.5s
290:	learn: 0.8402684	test: 0.8353556	best: 0.8353556 (290)	total: 10.4s	remaining: 25.4s
291:	learn: 0.8403144	test: 0.8353876	best: 0.8353876 (291)	total: 10.5s	remaining: 25.4s
292:	learn: 0.8403377	test: 0.8354150	best: 0.8354150 (292)	total: 10.5s	remaining: 25.3s
293:	learn: 0.8403387	test: 0.8354127	best: 0.8354150 (292)	total: 10.5s	remaining: 25.3s
294:	learn: 0.8403809	test: 0.8354106	best: 0.8354150 (292)	total: 10.6s	remaining: 25.2s
295:	learn: 0.8404376	test: 0.8354201	best: 0.8354201 (295)	total: 10.6s	remaining: 25.2s
296:	learn

379:	learn: 0.8426149	test: 0.8358909	best: 0.8358909 (379)	total: 13.6s	remaining: 22.2s
380:	learn: 0.8426150	test: 0.8358909	best: 0.8358909 (379)	total: 13.6s	remaining: 22.2s
381:	learn: 0.8426510	test: 0.8359121	best: 0.8359121 (381)	total: 13.7s	remaining: 22.1s
382:	learn: 0.8426632	test: 0.8359175	best: 0.8359175 (382)	total: 13.7s	remaining: 22.1s
383:	learn: 0.8426684	test: 0.8359185	best: 0.8359185 (383)	total: 13.7s	remaining: 22s
384:	learn: 0.8426806	test: 0.8359330	best: 0.8359330 (384)	total: 13.8s	remaining: 22s
385:	learn: 0.8427129	test: 0.8359110	best: 0.8359330 (384)	total: 13.8s	remaining: 22s
386:	learn: 0.8427613	test: 0.8359236	best: 0.8359330 (384)	total: 13.9s	remaining: 21.9s
387:	learn: 0.8427827	test: 0.8359283	best: 0.8359330 (384)	total: 13.9s	remaining: 21.9s
388:	learn: 0.8428225	test: 0.8359383	best: 0.8359383 (388)	total: 13.9s	remaining: 21.9s
389:	learn: 0.8428330	test: 0.8359369	best: 0.8359383 (388)	total: 14s	remaining: 21.8s
390:	learn: 0.8428

...

177:	learn: 0.8358264	test: 0.8361442	best: 0.8361442 (177)	total: 6.11s	remaining: 28.2s
178:	learn: 0.8358826	test: 0.8362009	best: 0.8362009 (178)	total: 6.14s	remaining: 28.2s
179:	learn: 0.8358708	test: 0.8361985	best: 0.8362009 (178)	total: 6.17s	remaining: 28.1s
180:	learn: 0.8359366	test: 0.8362215	best: 0.8362215 (180)	total: 6.21s	remaining: 28.1s
181:	learn: 0.8360577	test: 0.8363262	best: 0.8363262 (181)	total: 6.24s	remaining: 28s
182:	learn: 0.8360665	test: 0.8363246	best: 0.8363262 (181)	total: 6.27s	remaining: 28s
183:	learn: 0.8361287	test: 0.8363584	best: 0.8363584 (183)	total: 6.3s	remaining: 27.9s
184:	learn: 0.8361991	test: 0.8364312	best: 0.8364312 (184)	total: 6.33s	remaining: 27.9s
185:	learn: 0.8362074	test: 0.8364339	best: 0.8364339 (185)	total: 6.37s	remaining: 27.9s
186:	learn: 0.8362074	test: 0.8364339	best: 0.8364339 (185)	total: 6.4s	remaining: 27.8s
187:	learn: 0.8362554	test: 0.8364483	best: 0.8364483 (187)	total: 6.42s	remaining: 27.8s
188:	learn: 0.83

272:	learn: 0.8385905	test: 0.8374862	best: 0.8374862 (272)	total: 9.1s	remaining: 24.2s
273:	learn: 0.8385975	test: 0.8374748	best: 0.8374862 (272)	total: 9.14s	remaining: 24.2s
274:	learn: 0.8386220	test: 0.8374958	best: 0.8374958 (274)	total: 9.17s	remaining: 24.2s
275:	learn: 0.8386692	test: 0.8375115	best: 0.8375115 (275)	total: 9.2s	remaining: 24.1s
276:	learn: 0.8386707	test: 0.8375115	best: 0.8375115 (275)	total: 9.23s	remaining: 24.1s
277:	learn: 0.8386988	test: 0.8375252	best: 0.8375252 (277)	total: 9.26s	remaining: 24s
278:	learn: 0.8386986	test: 0.8375262	best: 0.8375262 (278)	total: 9.29s	remaining: 24s
279:	learn: 0.8387171	test: 0.8375460	best: 0.8375460 (279)	total: 9.33s	remaining: 24s
280:	learn: 0.8387516	test: 0.8375628	best: 0.8375628 (280)	total: 9.37s	remaining: 24s
281:	learn: 0.8387512	test: 0.8375635	best: 0.8375635 (281)	total: 9.39s	remaining: 23.9s
282:	learn: 0.8387556	test: 0.8375613	best: 0.8375635 (281)	total: 9.43s	remaining: 23.9s
283:	learn: 0.838772

367:	learn: 0.8409800	test: 0.8384692	best: 0.8384692 (367)	total: 12.7s	remaining: 21.7s
368:	learn: 0.8410079	test: 0.8384797	best: 0.8384797 (368)	total: 12.7s	remaining: 21.7s
369:	learn: 0.8410255	test: 0.8384825	best: 0.8384825 (369)	total: 12.7s	remaining: 21.7s
370:	learn: 0.8410560	test: 0.8384791	best: 0.8384825 (369)	total: 12.8s	remaining: 21.7s
371:	learn: 0.8410826	test: 0.8385144	best: 0.8385144 (371)	total: 12.8s	remaining: 21.6s
372:	learn: 0.8410848	test: 0.8385133	best: 0.8385144 (371)	total: 12.9s	remaining: 21.6s
373:	learn: 0.8411129	test: 0.8385179	best: 0.8385179 (373)	total: 12.9s	remaining: 21.6s
374:	learn: 0.8411416	test: 0.8385160	best: 0.8385179 (373)	total: 12.9s	remaining: 21.6s
375:	learn: 0.8411416	test: 0.8385160	best: 0.8385179 (373)	total: 13s	remaining: 21.5s
376:	learn: 0.8411766	test: 0.8385135	best: 0.8385179 (373)	total: 13s	remaining: 21.5s
377:	learn: 0.8412068	test: 0.8385141	best: 0.8385179 (373)	total: 13s	remaining: 21.5s
378:	learn: 0.84

462:	learn: 0.8434786	test: 0.8391875	best: 0.8392054 (459)	total: 16.6s	remaining: 19.3s
463:	learn: 0.8435076	test: 0.8391819	best: 0.8392054 (459)	total: 16.7s	remaining: 19.3s
464:	learn: 0.8435201	test: 0.8391845	best: 0.8392054 (459)	total: 16.7s	remaining: 19.2s
465:	learn: 0.8435324	test: 0.8391746	best: 0.8392054 (459)	total: 16.8s	remaining: 19.2s
466:	learn: 0.8435685	test: 0.8391631	best: 0.8392054 (459)	total: 16.8s	remaining: 19.2s
467:	learn: 0.8435839	test: 0.8391693	best: 0.8392054 (459)	total: 16.8s	remaining: 19.1s
468:	learn: 0.8436146	test: 0.8391827	best: 0.8392054 (459)	total: 16.9s	remaining: 19.1s
469:	learn: 0.8436249	test: 0.8391724	best: 0.8392054 (459)	total: 16.9s	remaining: 19.1s
470:	learn: 0.8436608	test: 0.8391618	best: 0.8392054 (459)	total: 16.9s	remaining: 19s
471:	learn: 0.8436698	test: 0.8391596	best: 0.8392054 (459)	total: 17s	remaining: 19s
472:	learn: 0.8437141	test: 0.8391506	best: 0.8392054 (459)	total: 17s	remaining: 19s
473:	learn: 0.843759

557:	learn: 0.8458995	test: 0.8393112	best: 0.8393275 (538)	total: 20.6s	remaining: 16.3s
558:	learn: 0.8459228	test: 0.8393163	best: 0.8393275 (538)	total: 20.7s	remaining: 16.3s
559:	learn: 0.8459321	test: 0.8393168	best: 0.8393275 (538)	total: 20.7s	remaining: 16.3s
560:	learn: 0.8459569	test: 0.8393346	best: 0.8393346 (560)	total: 20.8s	remaining: 16.2s
561:	learn: 0.8459782	test: 0.8393316	best: 0.8393346 (560)	total: 20.8s	remaining: 16.2s
562:	learn: 0.8459838	test: 0.8393314	best: 0.8393346 (560)	total: 20.8s	remaining: 16.2s
563:	learn: 0.8460144	test: 0.8393226	best: 0.8393346 (560)	total: 20.9s	remaining: 16.1s
564:	learn: 0.8460429	test: 0.8393208	best: 0.8393346 (560)	total: 20.9s	remaining: 16.1s
565:	learn: 0.8460732	test: 0.8393028	best: 0.8393346 (560)	total: 20.9s	remaining: 16s
566:	learn: 0.8460907	test: 0.8393048	best: 0.8393346 (560)	total: 20.9s	remaining: 16s
567:	learn: 0.8461041	test: 0.8392939	best: 0.8393346 (560)	total: 21s	remaining: 16s
568:	learn: 0.8461

# Summary


- Boosting algorithms are the best for heterogeneous data.
- Lightgbm is the fastest and usually the most accurate;
- CatBoost doesn't need a lot of tuning;
- CatBoost shows the best result on data with many categorical variables;
- Always try three methods and ensemble them;
- Remember about regularization and comparing your training score with validation;
- Use LightGBM for experiments and in the end execute other algorithms;
- Firstly, set default parameters, do feature engineering and then come back to parameters tuning;
- More practice will give you more understanding and intuition;
- Use magic of boosting in real life :)



# Sources:

- Открытый курс машинного обучения. Тема 10. Градиентный бустинг. Часть 1 : https://habrahabr.ru/company/ods/blog/327250/
- Introduction to Boosted Trees: http://xgboost.readthedocs.io/en/latest/model.html
- XGBoost, a Top Machine Learning Method on Kaggle, Explained : https://www.kdnuggets.com/2017/10/xgboost-top-machine-learning-method-kaggle-explained.html
- LightGBM: A Highly Efficient Gradient Boosting Decision Tree: https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf
- CatBoost: A machine learning library to handle categorical (CAT) data automatically: https://www.analyticsvidhya.com/blog/2017/08/catboost-automated-categorical-data/
- CatBoost: https://tech.yandex.com/catboost